In [1]:
import tensorflow as tf
import numpy as np

print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPU Devices:", tf.config.list_physical_devices('GPU'))

# Additional GPU check
from tensorflow.python.client import device_lib
print("\nAll available devices:")
for device in device_lib.list_local_devices():
    print(f"  {device.name}: {device.device_type}")

TensorFlow version: 2.10.1
NumPy version: 1.26.4
Num GPUs Available: 1
GPU Devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

All available devices:
  /device:CPU:0: CPU
  /device:GPU:0: GPU


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime

class SimpleRNNDefectDetector:
    def __init__(self, dataset_path, img_height=128, img_width=128, sequence_length=16):
        """
        Initialize SimpleRNN Defect Detection Model
        
        Args:
            dataset_path: Path to final_merged_dataset
            img_height: Height to resize images
            img_width: Width to resize images
            sequence_length: Number of feature vectors to treat as sequence
        """
        self.dataset_path = Path(dataset_path)
        self.img_height = img_height
        self.img_width = img_width
        self.sequence_length = sequence_length
        self.model = None
        self.history = None
        self.class_names = []
        self.num_classes = 0
        
        # Create output directories
        self.output_dir = Path("rnn_output")
        self.model_dir = self.output_dir / "models"
        self.plots_dir = self.output_dir / "plots"
        self.logs_dir = self.output_dir / "logs"
        
        for dir_path in [self.output_dir, self.model_dir, self.plots_dir, self.logs_dir]:
            dir_path.mkdir(exist_ok=True)
    
    def prepare_data(self, validation_split=0.2, test_split=0.1, batch_size=32):
        """
        Prepare dataset with train/validation/test splits
        """
        print("\n" + "="*60)
        print("DATA PREPARATION")
        print("="*60)
        
        # Get all image paths and labels
        image_paths = []
        labels = []
        
        print("\n📁 Scanning dataset...")
        for class_dir in self.dataset_path.rglob("*"):
            if class_dir.is_dir():
                images = list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + \
                         list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp"))
                
                if images:
                    class_name = class_dir.name
                    if class_name not in self.class_names:
                        self.class_names.append(class_name)
                    
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_name)
        
        self.num_classes = len(self.class_names)
        print(f"✓ Found {len(image_paths)} images across {self.num_classes} classes")
        print(f"✓ Classes: {self.class_names[:10]}{'...' if len(self.class_names) > 10 else ''}")
        
        # Convert labels to integers
        label_to_index = {label: idx for idx, label in enumerate(self.class_names)}
        labels_encoded = [label_to_index[label] for label in labels]
        
        # Split data
        X_temp, X_test, y_temp, y_test = train_test_split(
            image_paths, labels_encoded, test_size=test_split, random_state=42, stratify=labels_encoded
        )
        
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=validation_split/(1-test_split), random_state=42, stratify=y_temp
        )
        
        print(f"\n✓ Train samples: {len(X_train)}")
        print(f"✓ Validation samples: {len(X_val)}")
        print(f"✓ Test samples: {len(X_test)}")
        
        # Create data generators
        self.train_generator = self._create_data_generator(X_train, y_train, batch_size, augment=True)
        self.val_generator = self._create_data_generator(X_val, y_val, batch_size, augment=False)
        self.test_generator = self._create_data_generator(X_test, y_test, batch_size, augment=False)
        
        self.steps_per_epoch = len(X_train) // batch_size
        self.validation_steps = len(X_val) // batch_size
        self.test_steps = len(X_test) // batch_size
        
        return len(X_train), len(X_val), len(X_test)
    
    def _create_data_generator(self, image_paths, labels, batch_size, augment=False):
        """Create a generator that yields batches of sequences"""
        def generator():
            indices = np.arange(len(image_paths))
            while True:
                np.random.shuffle(indices)
                
                for start_idx in range(0, len(indices) - batch_size + 1, batch_size):
                    batch_indices = indices[start_idx:start_idx + batch_size]
                    
                    batch_images = []
                    batch_labels = []
                    
                    for idx in batch_indices:
                        # Load and preprocess image
                        img = tf.keras.preprocessing.image.load_img(
                            image_paths[idx],
                            target_size=(self.img_height, self.img_width)
                        )
                        img_array = tf.keras.preprocessing.image.img_to_array(img)
                        img_array = img_array / 255.0  # Normalize
                        
                        # Data augmentation
                        if augment:
                            img_array = self._augment_image(img_array)
                        
                        batch_images.append(img_array)
                        batch_labels.append(labels[idx])
                    
                    # Convert to sequences for RNN
                    batch_images = np.array(batch_images)
                    batch_labels = np.array(batch_labels)
                    
                    # Reshape images to sequences: flatten spatial dimensions and create sequences
                    # Shape: (batch, height, width, channels) -> (batch, sequence_length, features)
                    sequences = self._image_to_sequence(batch_images)
                    
                    yield sequences, tf.keras.utils.to_categorical(batch_labels, self.num_classes)
        
        return generator()
    
    def _augment_image(self, img_array):
        """Apply random augmentations to image"""
        # Random flip
        if np.random.random() > 0.5:
            img_array = tf.image.flip_left_right(img_array)
        
        # Random brightness
        img_array = tf.image.random_brightness(img_array, max_delta=0.2)
        
        # Random contrast
        img_array = tf.image.random_contrast(img_array, lower=0.8, upper=1.2)
        
        return tf.clip_by_value(img_array, 0.0, 1.0)
    
    def _image_to_sequence(self, images):
        """
        Convert images to sequences for RNN processing
        Split image into horizontal strips to create a sequence
        """
        batch_size = images.shape[0]
        
        # Split image into sequence_length horizontal strips
        strip_height = self.img_height // self.sequence_length
        sequences = []
        
        for i in range(batch_size):
            img = images[i]
            strips = []
            
            for j in range(self.sequence_length):
                start_h = j * strip_height
                end_h = start_h + strip_height
                strip = img[start_h:end_h, :, :]
                # Flatten strip to create feature vector
                strip_flat = strip.reshape(-1)
                strips.append(strip_flat)
            
            sequences.append(strips)
        
        return np.array(sequences)
    
    def build_model(self, rnn_units=128, dropout_rate=0.3, learning_rate=0.001):
        """
        Build SimpleRNN model architecture
        """
        print("\n" + "="*60)
        print("BUILDING SIMPLERNN MODEL")
        print("="*60)
        
        # Calculate input feature size
        strip_height = self.img_height // self.sequence_length
        feature_size = strip_height * self.img_width * 3  # 3 for RGB channels
        
        model = keras.Sequential([
            # Input layer
            layers.Input(shape=(self.sequence_length, feature_size)),
            
            # First SimpleRNN layer with return sequences
            layers.SimpleRNN(rnn_units, return_sequences=True, activation='tanh'),
            layers.Dropout(dropout_rate),
            
            # Second SimpleRNN layer
            layers.SimpleRNN(rnn_units // 2, activation='tanh'),
            layers.Dropout(dropout_rate),
            
            # Dense layers
            layers.Dense(256, activation='relu'),
            layers.Dropout(dropout_rate),
            
            layers.Dense(128, activation='relu'),
            layers.Dropout(dropout_rate / 2),
            
            # Output layer
            layers.Dense(self.num_classes, activation='softmax')
        ])
        
        # Compile model
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Model architecture:")
        model.summary()
        
        # Save model architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def train(self, epochs=50, early_stopping_patience=10):
        """
        Train the SimpleRNN model
        """
        print("\n" + "="*60)
        print("TRAINING SIMPLERNN MODEL")
        print("="*60)
        
        if self.model is None:
            raise ValueError("Model not built. Call build_model() first.")
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ModelCheckpoint(
                filepath=str(self.model_dir / 'best_model.h5'),
                monitor='val_accuracy',
                save_best_only=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            keras.callbacks.CSVLogger(
                str(self.logs_dir / 'training_log.csv')
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Train model
        self.history = self.model.fit(
            self.train_generator,
            steps_per_epoch=self.steps_per_epoch,
            epochs=epochs,
            validation_data=self.val_generator,
            validation_steps=self.validation_steps,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"\n✓ Training completed!")
        print(f"⏱️  End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save final model
        self.model.save(self.model_dir / 'final_model.h5')
        print(f"✓ Model saved to: {self.model_dir}")
        
        return self.history
    
    def evaluate(self):
        """
        Evaluate model on test set
        """
        print("\n" + "="*60)
        print("EVALUATING MODEL")
        print("="*60)
        
        if self.model is None:
            raise ValueError("Model not trained. Call train() first.")
        
        print("\n📊 Evaluating on test set...")
        test_results = self.model.evaluate(
            self.test_generator,
            steps=self.test_steps,
            verbose=1
        )
        
        print("\n✓ Test Results:")
        print(f"  Loss: {test_results[0]:.4f}")
        print(f"  Accuracy: {test_results[1]:.4f}")
        print(f"  Top-5 Accuracy: {test_results[2]:.4f}")
        
        # Save results
        results = {
            'test_loss': float(test_results[0]),
            'test_accuracy': float(test_results[1]),
            'test_top5_accuracy': float(test_results[2]),
            'num_classes': self.num_classes,
            'evaluation_time': datetime.now().isoformat()
        }
        
        with open(self.logs_dir / 'test_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        return results
    
    def plot_training_history(self):
        """
        Plot training history
        """
        print("\n📈 Generating training plots...")
        
        if self.history is None:
            print("⚠️  No training history available")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot accuracy
        axes[0, 0].plot(self.history.history['accuracy'], label='Train Accuracy')
        axes[0, 0].plot(self.history.history['val_accuracy'], label='Val Accuracy')
        axes[0, 0].set_title('Model Accuracy')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Accuracy')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
        
        # Plot loss
        axes[0, 1].plot(self.history.history['loss'], label='Train Loss')
        axes[0, 1].plot(self.history.history['val_loss'], label='Val Loss')
        axes[0, 1].set_title('Model Loss')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Loss')
        axes[0, 1].legend()
        axes[0, 1].grid(True)
        
        # Plot top-5 accuracy
        axes[1, 0].plot(self.history.history['top_5_accuracy'], label='Train Top-5 Acc')
        axes[1, 0].plot(self.history.history['val_top_5_accuracy'], label='Val Top-5 Acc')
        axes[1, 0].set_title('Top-5 Accuracy')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Accuracy')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
        
        # Plot learning rate (if available)
        if 'lr' in self.history.history:
            axes[1, 1].plot(self.history.history['lr'])
            axes[1, 1].set_title('Learning Rate')
            axes[1, 1].set_xlabel('Epoch')
            axes[1, 1].set_ylabel('Learning Rate')
            axes[1, 1].set_yscale('log')
            axes[1, 1].grid(True)
        else:
            axes[1, 1].text(0.5, 0.5, 'Learning Rate\nNot Tracked', 
                           ha='center', va='center', fontsize=14)
            axes[1, 1].axis('off')
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'training_history.png', dpi=300, bbox_inches='tight')
        print(f"✓ Training plots saved to: {self.plots_dir / 'training_history.png'}")
        plt.close()
    
    def save_training_summary(self):
        """
        Save comprehensive training summary
        """
        print("\n📝 Generating training summary...")
        
        summary_file = self.logs_dir / 'training_summary.txt'
        
        with open(summary_file, 'w', encoding='utf-8') as f:
            f.write("="*70 + "\n")
            f.write("SIMPLERNN DEFECT DETECTION - TRAINING SUMMARY\n")
            f.write("="*70 + "\n\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            f.write("-"*70 + "\n")
            f.write("DATASET INFORMATION\n")
            f.write("-"*70 + "\n")
            f.write(f"Dataset Path: {self.dataset_path}\n")
            f.write(f"Number of Classes: {self.num_classes}\n")
            f.write(f"Image Size: {self.img_height}x{self.img_width}\n")
            f.write(f"Sequence Length: {self.sequence_length}\n\n")
            
            f.write("-"*70 + "\n")
            f.write("MODEL CONFIGURATION\n")
            f.write("-"*70 + "\n")
            f.write("Architecture: SimpleRNN with Dense layers\n")
            f.write(f"Total Parameters: {self.model.count_params():,}\n\n")
            
            if self.history:
                f.write("-"*70 + "\n")
                f.write("TRAINING RESULTS\n")
                f.write("-"*70 + "\n")
                f.write(f"Epochs Trained: {len(self.history.history['loss'])}\n")
                f.write(f"Best Train Accuracy: {max(self.history.history['accuracy']):.4f}\n")
                f.write(f"Best Val Accuracy: {max(self.history.history['val_accuracy']):.4f}\n")
                f.write(f"Final Train Loss: {self.history.history['loss'][-1]:.4f}\n")
                f.write(f"Final Val Loss: {self.history.history['val_loss'][-1]:.4f}\n\n")
            
            f.write("="*70 + "\n")
        
        print(f"✓ Training summary saved to: {summary_file}")


def main():
    """
    Main execution function
    """
    print("\n" + "="*60)
    print("SIMPLERNN DEFECT DETECTION PIPELINE")
    print("="*60)
    
    # Configuration
    DATASET_PATH = "final_merged_dataset"
    IMG_HEIGHT = 128
    IMG_WIDTH = 128
    SEQUENCE_LENGTH = 16
    BATCH_SIZE = 32
    EPOCHS = 50
    RNN_UNITS = 128
    DROPOUT_RATE = 0.3
    LEARNING_RATE = 0.001
    
    # Initialize detector
    detector = SimpleRNNDefectDetector(
        dataset_path=DATASET_PATH,
        img_height=IMG_HEIGHT,
        img_width=IMG_WIDTH,
        sequence_length=SEQUENCE_LENGTH
    )
    
    # Prepare data
    train_size, val_size, test_size = detector.prepare_data(
        validation_split=0.2,
        test_split=0.1,
        batch_size=BATCH_SIZE
    )
    
    # Build model
    detector.build_model(
        rnn_units=RNN_UNITS,
        dropout_rate=DROPOUT_RATE,
        learning_rate=LEARNING_RATE
    )
    
    # Train model
    detector.train(epochs=EPOCHS, early_stopping_patience=10)
    
    # Evaluate model
    test_results = detector.evaluate()
    
    # Plot training history
    detector.plot_training_history()
    
    # Save summary
    detector.save_training_summary()
    
    print("\n" + "="*60)
    print("PIPELINE COMPLETED SUCCESSFULLY!")
    print("="*60)
    print(f"\n📁 Output Directory: rnn_output/")
    print(f"📊 Models saved in: rnn_output/models/")
    print(f"📈 Plots saved in: rnn_output/plots/")
    print(f"📝 Logs saved in: rnn_output/logs/")
    print("\n" + "="*60)


if __name__ == "__main__":
    main()


SIMPLERNN DEFECT DETECTION PIPELINE

DATA PREPARATION

📁 Scanning dataset...
✓ Found 9808 images across 63 classes
✓ Classes: ['broken_large', 'broken_small', 'contamination', 'good', 'bent_wire', 'cable_swap', 'combined', 'cut_inner_insulation', 'cut_outer_insulation', 'missing_cable']...

✓ Train samples: 6865
✓ Validation samples: 1962
✓ Test samples: 981

BUILDING SIMPLERNN MODEL

✓ Model architecture:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 16, 128)           409728    
                                                                 
 dropout (Dropout)           (None, 16, 128)           0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                12352     
                                                                 
 dropout_1 (Dropout)         (None, 64)  

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [30]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

class DefectDetectionModel:
    """Base class for defect detection models"""
    
    def __init__(self, dataset_path, model_name, img_height=224, img_width=224):
        self.dataset_path = Path(dataset_path)
        self.model_name = model_name
        self.img_height = img_height
        self.img_width = img_width
        self.model = None
        self.history = None
        self.class_names = []
        self.num_classes = 0
        
        # Create output directories
        self.output_dir = Path(f"{model_name}_output")
        self.model_dir = self.output_dir / "models"
        self.plots_dir = self.output_dir / "plots"
        self.logs_dir = self.output_dir / "logs"
        
        for dir_path in [self.output_dir, self.model_dir, self.plots_dir, self.logs_dir]:
            dir_path.mkdir(exist_ok=True)
    
    def prepare_data(self, validation_split=0.2, test_split=0.1, batch_size=32):
        """Prepare dataset with efficient TensorFlow pipeline"""
        print("\n" + "="*70)
        print(f"DATA PREPARATION - {self.model_name}")
        print("="*70)
        
        # Get all image paths and labels
        image_paths = []
        labels = []
        
        print("\n📁 Scanning dataset...")
        for class_dir in self.dataset_path.rglob("*"):
            if class_dir.is_dir():
                images = (list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + 
                         list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp")))
                
                if images:
                    class_name = class_dir.name
                    if class_name not in self.class_names:
                        self.class_names.append(class_name)
                    
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_name)
        
        self.class_names.sort()  # Sort for consistency
        self.num_classes = len(self.class_names)
        print(f"✓ Found {len(image_paths)} images")
        print(f"✓ Number of classes: {self.num_classes}")
        print(f"✓ Classes: {self.class_names[:10]}{'...' if len(self.class_names) > 10 else ''}")
        
        # Create label mapping
        label_to_index = {label: idx for idx, label in enumerate(self.class_names)}
        labels_encoded = [label_to_index[label] for label in labels]
        
        # Create tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels_encoded))
        dataset = dataset.shuffle(len(image_paths), seed=42)
        
        # Calculate splits
        train_size = int(len(image_paths) * (1 - validation_split - test_split))
        val_size = int(len(image_paths) * validation_split)
        test_size = len(image_paths) - train_size - val_size
        
        # Split datasets
        train_ds = dataset.take(train_size)
        val_ds = dataset.skip(train_size).take(val_size)
        test_ds = dataset.skip(train_size + val_size)
        
        # Process datasets
        self.train_dataset = train_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=True),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.val_dataset = val_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.test_dataset = test_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        print(f"\n✓ Train samples: {train_size}")
        print(f"✓ Validation samples: {val_size}")
        print(f"✓ Test samples: {test_size}")
        
        return train_size, val_size, test_size
    
    def _load_and_preprocess(self, image_path, label, augment=False):
        """Load and preprocess image"""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, [self.img_height, self.img_width])
        
        # Data augmentation
        if augment:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            
            # Random rotation
            img = tf.image.rot90(img, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
        
        img = tf.clip_by_value(img, 0.0, 255.0)
        
        # Don't normalize here - let the model handle it
        # This avoids serialization issues
        
        label_onehot = tf.one_hot(label, self.num_classes)
        return img, label_onehot
    
    def _normalize_image(self, img):
        """Normalize image - override in subclasses if needed"""
        return img  # Keep raw [0, 255] values
    
    def train(self, epochs=50, early_stopping_patience=10):
        """Train the model"""
        print("\n" + "="*70)
        print(f"TRAINING {self.model_name}")
        print("="*70)
        
        if self.model is None:
            raise ValueError("Model not built. Call build_model() first.")
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ModelCheckpoint(
                filepath=str(self.model_dir / 'best_model.h5'),
                monitor='val_accuracy',
                save_best_only=True,
                save_weights_only=False,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            keras.callbacks.CSVLogger(
                str(self.logs_dir / 'training_log.csv')
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Train model
        self.history = self.model.fit(
            self.train_dataset,
            epochs=epochs,
            validation_data=self.val_dataset,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"\n✓ Training completed!")
        print(f"⏱️  End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save final model using SavedModel format
        try:
            self.model.save(self.model_dir / 'final_model.h5')
            print(f"✓ Model saved to: {self.model_dir / 'final_model.h5'}")
        except:
            # If H5 format fails, use SavedModel format
            self.model.save(str(self.model_dir / 'final_model'))
            print(f"✓ Model saved to: {self.model_dir / 'final_model'}")
        
        return self.history
    
    def evaluate(self):
        """Evaluate model on test set"""
        print("\n" + "="*70)
        print(f"EVALUATING {self.model_name}")
        print("="*70)
        
        print("\n📊 Evaluating on test set...")
        test_results = self.model.evaluate(self.test_dataset, verbose=1)
        
        print("\n✓ Test Results:")
        print(f"  Loss: {test_results[0]:.4f}")
        print(f"  Accuracy: {test_results[1]:.4f}")
        if len(test_results) > 2:
            print(f"  Top-5 Accuracy: {test_results[2]:.4f}")
        
        # Get predictions for confusion matrix
        print("\n📊 Generating detailed metrics...")
        y_true = []
        y_pred = []
        
        for images, labels in self.test_dataset:
            predictions = self.model.predict(images, verbose=0)
            y_true.extend(np.argmax(labels.numpy(), axis=1))
            y_pred.extend(np.argmax(predictions, axis=1))
        
        # Save results
        results = {
            'test_loss': float(test_results[0]),
            'test_accuracy': float(test_results[1]),
            'num_classes': self.num_classes,
            'evaluation_time': datetime.now().isoformat()
        }
        
        if len(test_results) > 2:
            results['test_top5_accuracy'] = float(test_results[2])
        
        with open(self.logs_dir / 'test_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        # Generate confusion matrix
        self._plot_confusion_matrix(y_true, y_pred)
        
        # Generate classification report
        self._save_classification_report(y_true, y_pred)
        
        return results
    
    def _plot_confusion_matrix(self, y_true, y_pred):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(max(10, self.num_classes), max(8, self.num_classes * 0.8)))
        
        # Normalize confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        sns.heatmap(cm_normalized, annot=False, cmap='Blues', 
                    xticklabels=self.class_names if self.num_classes <= 20 else False,
                    yticklabels=self.class_names if self.num_classes <= 20 else False)
        
        plt.title(f'{self.model_name} - Confusion Matrix (Normalized)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Confusion matrix saved to: {self.plots_dir / 'confusion_matrix.png'}")
    
    def _save_classification_report(self, y_true, y_pred):
        """Save classification report"""
        report = classification_report(y_true, y_pred, target_names=self.class_names, 
                                      zero_division=0)
        
        with open(self.logs_dir / 'classification_report.txt', 'w') as f:
            f.write(f"{self.model_name} - Classification Report\n")
            f.write("="*70 + "\n\n")
            f.write(report)
        
        print(f"✓ Classification report saved to: {self.logs_dir / 'classification_report.txt'}")
    
    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            print("⚠️  No training history available")
            return
        
        print("\n📈 Generating training plots...")
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot accuracy
        axes[0].plot(self.history.history['accuracy'], label='Train Accuracy', linewidth=2)
        axes[0].plot(self.history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
        axes[0].set_title(f'{self.model_name} - Model Accuracy', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Plot loss
        axes[1].plot(self.history.history['loss'], label='Train Loss', linewidth=2)
        axes[1].plot(self.history.history['val_loss'], label='Val Loss', linewidth=2)
        axes[1].set_title(f'{self.model_name} - Model Loss', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'training_history.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Training plots saved to: {self.plots_dir / 'training_history.png'}")


In [31]:
class TensorSafeCSVLogger(keras.callbacks.Callback):
    """Ensures metrics are converted to float before CSVLogger writes them"""
    def __init__(self, filename):
        super().__init__()
        self.csv_logger = keras.callbacks.CSVLogger(filename)

    def on_epoch_end(self, epoch, logs=None):
        safe_logs = {}
        for k, v in logs.items():
            if hasattr(v, "numpy"):
                safe_logs[k] = float(v.numpy())
            elif isinstance(v, (np.ndarray, list)):
                safe_logs[k] = float(np.mean(v))
            else:
                safe_logs[k] = float(v)
        self.csv_logger.on_epoch_end(epoch, safe_logs)

    def on_train_end(self, logs=None):
        self.csv_logger.on_train_end(logs)


In [5]:
class EfficientNetDefectDetector(DefectDetectionModel):
    """EfficientNet-B0 for defect detection"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224):
        super().__init__(dataset_path, "EfficientNetB0", img_height, img_width)
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.3, fine_tune_layers=50):
        """Build EfficientNet model with transfer learning"""
        print("\n" + "="*70)
        print("BUILDING EFFICIENTNET MODEL")
        print("="*70)
        
        # Load pre-trained EfficientNet
        base_model = EfficientNetB0(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Freeze base model initially
        base_model.trainable = False
        
        # Build model (EfficientNet expects [0, 255] range)
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        x = base_model(inputs, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Base model frozen: {not base_model.trainable}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def unfreeze_and_fine_tune(self, fine_tune_layers=50, learning_rate=0.0001):
        """Unfreeze top layers for fine-tuning"""
        print(f"\n🔓 Unfreezing top {fine_tune_layers} layers for fine-tuning...")
        
        base_model = self.model.layers[1]  # Get EfficientNet base
        base_model.trainable = True
        
        # Freeze all layers except the last fine_tune_layers
        for layer in base_model.layers[:-fine_tune_layers]:
            layer.trainable = False
        
        # Recompile with lower learning rate
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        print(f"✓ Fine-tuning enabled with learning rate: {learning_rate}")


In [ ]:
class ResNet50DefectDetector(DefectDetectionModel):
    """ResNet-50 for defect detection"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224):
        super().__init__(dataset_path, "ResNet50", img_height, img_width)
    
    def _normalize_image(self, img):
        """ResNet uses standard normalization"""
        return img / 255.0
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.3):
        """Build ResNet-50 model with transfer learning"""
        print("\n" + "="*70)
        print("BUILDING RESNET-50 MODEL")
        print("="*70)
        
        # Load pre-trained ResNet50
        base_model = ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Freeze base model initially
        base_model.trainable = False
        
        # Build model with preprocessing
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # Add preprocessing for ResNet (ImageNet normalization)
        x = layers.Rescaling(scale=1./255)(inputs)
        x = layers.Normalization(
            mean=[0.485, 0.456, 0.406],
            variance=[0.229**2, 0.224**2, 0.225**2]
        )(x)
        
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Base model frozen: {not base_model.trainable}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def unfreeze_and_fine_tune(self, fine_tune_from_layer=140, learning_rate=0.0001):
        """Unfreeze layers for fine-tuning"""
        print(f"\n🔓 Unfreezing layers from layer {fine_tune_from_layer}...")
        
        base_model = self.model.layers[1]  # Get ResNet base
        base_model.trainable = True
        
        # Freeze layers before fine_tune_from_layer
        for layer in base_model.layers[:fine_tune_from_layer]:
            layer.trainable = False
        
        # Recompile
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        print(f"✓ Fine-tuning enabled with learning rate: {learning_rate}")


In [ ]:
class VisionTransformerDefectDetector(DefectDetectionModel):
    """Vision Transformer (ViT) for defect detection"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224, patch_size=16):
        super().__init__(dataset_path, "VisionTransformer", img_height, img_width)
        self.patch_size = patch_size
        self.num_patches = (img_height // patch_size) * (img_width // patch_size)
    
    def build_model(self, transformer_layers=8, num_heads=8, mlp_dim=2048, 
                    dropout_rate=0.1, learning_rate=0.001):
        """Build Vision Transformer model"""
        print("\n" + "="*70)
        print("BUILDING VISION TRANSFORMER MODEL")
        print("="*70)
        
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # Normalize images to [0, 1]
        x = layers.Rescaling(scale=1./255)(inputs)
        
        # Create patches
        patches = self._create_patches(x)
        
        # Patch embedding
        projection_dim = 768
        patch_embeddings = layers.Dense(projection_dim)(patches)
        
        # Position embedding
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        position_embeddings = layers.Embedding(
            input_dim=self.num_patches, output_dim=projection_dim
        )(positions)
        
        # Add position embeddings to patch embeddings
        encoded = patch_embeddings + position_embeddings
        
        # Transformer blocks
        for _ in range(transformer_layers):
            # Layer normalization 1
            x1 = layers.LayerNormalization(epsilon=1e-6)(encoded)
            
            # Multi-head attention
            attention_output = layers.MultiHeadAttention(
                num_heads=num_heads, key_dim=projection_dim // num_heads, dropout=dropout_rate
            )(x1, x1)
            
            # Skip connection 1
            x2 = layers.Add()([attention_output, encoded])
            
            # Layer normalization 2
            x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
            
            # MLP
            x3 = layers.Dense(mlp_dim, activation='gelu')(x3)
            x3 = layers.Dropout(dropout_rate)(x3)
            x3 = layers.Dense(projection_dim)(x3)
            x3 = layers.Dropout(dropout_rate)(x3)
            
            # Skip connection 2
            encoded = layers.Add()([x3, x2])
        
        # Final layer normalization
        representation = layers.LayerNormalization(epsilon=1e-6)(encoded)
        
        # Global average pooling
        representation = layers.GlobalAveragePooling1D()(representation)
        
        # Classification head
        representation = layers.Dropout(dropout_rate)(representation)
        features = layers.Dense(mlp_dim, activation='gelu')(representation)
        features = layers.Dropout(dropout_rate)(features)
        outputs = layers.Dense(self.num_classes, activation='softmax')(features)
        
        # Create model
        model = keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Vision Transformer built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Number of patches: {self.num_patches}")
        print(f"✓ Transformer layers: {transformer_layers}")
        print(f"✓ Attention heads: {num_heads}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def _create_patches(self, images):
        """Extract patches from images"""
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
        return patches
 

In [19]:
def compare_models(results_dict):
    """Compare results from different models"""
    print("\n" + "="*70)
    print("MODEL COMPARISON")
    print("="*70)
    
    comparison_data = []
    for model_name, results in results_dict.items():
        comparison_data.append({
            'Model': model_name,
            'Test Accuracy': f"{results['test_accuracy']*100:.2f}%",
            'Test Loss': f"{results['test_loss']:.4f}",
            'Top-5 Accuracy': f"{results.get('test_top5_accuracy', 0)*100:.2f}%" if 'test_top5_accuracy' in results else 'N/A'
        })
    
    print("\n📊 Performance Comparison:")
    print("-" * 70)
    for data in comparison_data:
        print(f"\n{data['Model']}:")
        print(f"  Accuracy: {data['Test Accuracy']}")
        print(f"  Loss: {data['Test Loss']}")
        print(f"  Top-5 Acc: {data['Top-5 Accuracy']}")
    
    # Save comparison
    with open('model_comparison.json', 'w') as f:
        json.dump(comparison_data, f, indent=2)
    
    print("\n✓ Comparison saved to: model_comparison.json")


def main():
    """Main execution function"""
    print("\n" + "="*70)
    print("ADVANCED DEFECT DETECTION PIPELINE")
    print("="*70)
    
    DATASET_PATH = "final_merged_dataset"
    EPOCHS = 30
    BATCH_SIZE = 32
    
    results = {}
    
    # 1. EfficientNet
    print("\n\n" + "🔷"*35)
    print("TRAINING EFFICIENTNET")
    print("🔷"*35)
    
    efficientnet = EfficientNetDefectDetector(DATASET_PATH)
    efficientnet.prepare_data(batch_size=BATCH_SIZE)
    efficientnet.build_model(learning_rate=0.001)
    efficientnet.train(epochs=EPOCHS)
    results['EfficientNet'] = efficientnet.evaluate()
    efficientnet.plot_training_history()
    
    # 2. ResNet-50
    print("\n\n" + "🔶"*35)
    print("TRAINING RESNET-50")
    print("🔶"*35)
    
    resnet = ResNet50DefectDetector(DATASET_PATH)
    resnet.prepare_data(batch_size=BATCH_SIZE)
    resnet.build_model(learning_rate=0.001)
    resnet.train(epochs=EPOCHS)
    results['ResNet50'] = resnet.evaluate()
    resnet.plot_training_history()
    
    # 3. Vision Transformer
    print("\n\n" + "🔹"*35)
    print("TRAINING VISION TRANSFORMER")
    print("🔹"*35)
    
    vit = VisionTransformerDefectDetector(DATASET_PATH)
    vit.prepare_data(batch_size=BATCH_SIZE)
    vit.build_model(transformer_layers=8, num_heads=8, learning_rate=0.001)
    vit.train(epochs=EPOCHS)
    results['VisionTransformer'] = vit.evaluate()
    vit.plot_training_history()
    
    # Compare all models
    compare_models(results)
    
    print("\n" + "="*70)
    print("🎉 ALL MODELS TRAINED SUCCESSFULLY!")
    print("="*70)
    print("\n📁 Output directories:")
    print("  - EfficientNetB0_output/")
    print("  - ResNet50_output/")
    print("  - VisionTransformer_output/")
    print("\n✓ Check model_comparison.json for detailed comparison")


if __name__ == "__main__":
    main()


ADVANCED DEFECT DETECTION PIPELINE


🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷
TRAINING EFFICIENTNET
🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

DATA PREPARATION - EfficientNetB0

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING EFFICIENTNET MODEL

✓ Model built successfully!
✓ Total parameters: 4,858,082
✓ Base model frozen: True

TRAINING EfficientNetB0

🚀 Starting training for 30 epochs...
⏱️  Start time: 2025-11-14 11:28:35

Epoch 1/30
215/215 [==============================] - ETA: 0s - loss: 1.5335 - accuracy: 0.6287 - top_5_accuracy: 0.8012
Epoch 1: val_accuracy improved from -inf to 0.67466, saving model to EfficientNetB0_output\models\best_model.h5


TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

class DefectDetectionModel:
    """Base class for defect detection models"""
    
    def __init__(self, dataset_path, model_name, img_height=224, img_width=224):
        self.dataset_path = Path(dataset_path)
        self.model_name = model_name
        self.img_height = img_height
        self.img_width = img_width
        self.model = None
        self.history = None
        self.class_names = []
        self.num_classes = 0
        
        # Create output directories
        self.output_dir = Path(f"{model_name}_output")
        self.model_dir = self.output_dir / "models"
        self.plots_dir = self.output_dir / "plots"
        self.logs_dir = self.output_dir / "logs"
        
        for dir_path in [self.output_dir, self.model_dir, self.plots_dir, self.logs_dir]:
            dir_path.mkdir(exist_ok=True)
    
    def prepare_data(self, validation_split=0.2, test_split=0.1, batch_size=32):
        """Prepare dataset with efficient TensorFlow pipeline"""
        print("\n" + "="*70)
        print(f"DATA PREPARATION - {self.model_name}")
        print("="*70)
        
        # Get all image paths and labels
        image_paths = []
        labels = []
        
        print("\n📁 Scanning dataset...")
        for class_dir in self.dataset_path.rglob("*"):
            if class_dir.is_dir():
                images = (list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + 
                         list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp")))
                
                if images:
                    class_name = class_dir.name
                    if class_name not in self.class_names:
                        self.class_names.append(class_name)
                    
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_name)
        
        self.class_names.sort()
        self.num_classes = len(self.class_names)
        print(f"✓ Found {len(image_paths)} images")
        print(f"✓ Number of classes: {self.num_classes}")
        print(f"✓ Classes: {self.class_names[:10]}{'...' if len(self.class_names) > 10 else ''}")
        
        # Create label mapping
        label_to_index = {label: idx for idx, label in enumerate(self.class_names)}
        labels_encoded = [label_to_index[label] for label in labels]
        
        # Create tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels_encoded))
        dataset = dataset.shuffle(len(image_paths), seed=42)
        
        # Calculate splits
        train_size = int(len(image_paths) * (1 - validation_split - test_split))
        val_size = int(len(image_paths) * validation_split)
        test_size = len(image_paths) - train_size - val_size
        
        # Split datasets
        train_ds = dataset.take(train_size)
        val_ds = dataset.skip(train_size).take(val_size)
        test_ds = dataset.skip(train_size + val_size)
        
        # Process datasets
        self.train_dataset = train_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=True),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.val_dataset = val_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.test_dataset = test_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        print(f"\n✓ Train samples: {train_size}")
        print(f"✓ Validation samples: {val_size}")
        print(f"✓ Test samples: {test_size}")
        
        return train_size, val_size, test_size
    
    def _load_and_preprocess(self, image_path, label, augment=False):
        """Load and preprocess image"""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, [self.img_height, self.img_width])
        
        # Data augmentation
        if augment:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.rot90(img, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
        
        img = tf.clip_by_value(img, 0.0, 255.0)
        
        # Normalize to [0, 1] for all models
        img = img / 255.0
        
        label_onehot = tf.one_hot(label, self.num_classes)
        return img, label_onehot
    
    def train(self, epochs=50, early_stopping_patience=10):
        """Train the model"""
        print("\n" + "="*70)
        print(f"TRAINING {self.model_name}")
        print("="*70)
        
        if self.model is None:
            raise ValueError("Model not built. Call build_model() first.")
        
        # Callbacks - SIMPLIFIED to avoid serialization issues
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            keras.callbacks.CSVLogger(
                str(self.logs_dir / 'training_log.csv')
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Train model
        self.history = self.model.fit(
            self.train_dataset,
            epochs=epochs,
            validation_data=self.val_dataset,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"\n✓ Training completed!")
        print(f"⏱️  End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save model weights only (avoids serialization issues)
        self.model.save_weights(str(self.model_dir / 'model_weights.h5'))
        print(f"✓ Model weights saved to: {self.model_dir / 'model_weights.h5'}")
        
        return self.history
    
    def evaluate(self):
        """Evaluate model on test set"""
        print("\n" + "="*70)
        print(f"EVALUATING {self.model_name}")
        print("="*70)
        
        print("\n📊 Evaluating on test set...")
        test_results = self.model.evaluate(self.test_dataset, verbose=1)
        
        print("\n✓ Test Results:")
        print(f"  Loss: {test_results[0]:.4f}")
        print(f"  Accuracy: {test_results[1]:.4f}")
        if len(test_results) > 2:
            print(f"  Top-5 Accuracy: {test_results[2]:.4f}")
        
        # Get predictions for confusion matrix
        print("\n📊 Generating detailed metrics...")
        y_true = []
        y_pred = []
        
        for images, labels in self.test_dataset:
            predictions = self.model.predict(images, verbose=0)
            y_true.extend(np.argmax(labels.numpy(), axis=1))
            y_pred.extend(np.argmax(predictions, axis=1))
        
        # Save results
        results = {
            'test_loss': float(test_results[0]),
            'test_accuracy': float(test_results[1]),
            'num_classes': self.num_classes,
            'evaluation_time': datetime.now().isoformat()
        }
        
        if len(test_results) > 2:
            results['test_top5_accuracy'] = float(test_results[2])
        
        with open(self.logs_dir / 'test_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        # Generate confusion matrix
        self._plot_confusion_matrix(y_true, y_pred)
        
        # Generate classification report
        self._save_classification_report(y_true, y_pred)
        
        return results
    
    def _plot_confusion_matrix(self, y_true, y_pred):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(max(10, self.num_classes), max(8, self.num_classes * 0.8)))
        
        # Normalize confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        sns.heatmap(cm_normalized, annot=False, cmap='Blues', 
                    xticklabels=self.class_names if self.num_classes <= 20 else False,
                    yticklabels=self.class_names if self.num_classes <= 20 else False)
        
        plt.title(f'{self.model_name} - Confusion Matrix (Normalized)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Confusion matrix saved to: {self.plots_dir / 'confusion_matrix.png'}")
    
    def _save_classification_report(self, y_true, y_pred):
        """Save classification report"""
        report = classification_report(y_true, y_pred, target_names=self.class_names, 
                                      zero_division=0)
        
        with open(self.logs_dir / 'classification_report.txt', 'w') as f:
            f.write(f"{self.model_name} - Classification Report\n")
            f.write("="*70 + "\n\n")
            f.write(report)
        
        print(f"✓ Classification report saved to: {self.logs_dir / 'classification_report.txt'}")
    
    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            print("⚠️  No training history available")
            return
        
        print("\n📈 Generating training plots...")
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot accuracy
        axes[0].plot(self.history.history['accuracy'], label='Train Accuracy', linewidth=2)
        axes[0].plot(self.history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
        axes[0].set_title(f'{self.model_name} - Model Accuracy', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Plot loss
        axes[1].plot(self.history.history['loss'], label='Train Loss', linewidth=2)
        axes[1].plot(self.history.history['val_loss'], label='Val Loss', linewidth=2)
        axes[1].set_title(f'{self.model_name} - Model Loss', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'training_history.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Training plots saved to: {self.plots_dir / 'training_history.png'}")


class EfficientNetDefectDetector(DefectDetectionModel):
    """EfficientNet-B0 for defect detection"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224):
        super().__init__(dataset_path, "EfficientNetB0", img_height, img_width)
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.3):
        """Build EfficientNet model with transfer learning"""
        print("\n" + "="*70)
        print("BUILDING EFFICIENTNET MODEL")
        print("="*70)
        
        # Load pre-trained EfficientNet (expects [0, 1] normalized images)
        base_model = EfficientNetB0(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Freeze base model initially
        base_model.trainable = False
        
        # Build model
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # Scale from [0,1] to [0,255] for EfficientNet
        x = layers.Rescaling(scale=255.0)(inputs)
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Base model frozen: {not base_model.trainable}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model


class ResNet50DefectDetector(DefectDetectionModel):
    """ResNet-50 for defect detection"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224):
        super().__init__(dataset_path, "ResNet50", img_height, img_width)
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.3):
        """Build ResNet-50 model with transfer learning"""
        print("\n" + "="*70)
        print("BUILDING RESNET-50 MODEL")
        print("="*70)
        
        # Load pre-trained ResNet50 with preprocessing included
        base_model = ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Freeze base model initially
        base_model.trainable = False
        
        # Build model - Input is [0, 1], ResNet preprocessing converts to [-1, 1]
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # Use Keras preprocessing for ResNet (handles normalization)
        x = tf.keras.applications.resnet.preprocess_input(inputs * 255.0)
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Base model frozen: {not base_model.trainable}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model




def compare_models(results_dict):
    """Compare results from different models"""
    print("\n" + "="*70)
    print("MODEL COMPARISON")
    print("="*70)
    
    comparison_data = []
    for model_name, results in results_dict.items():
        comparison_data.append({
            'Model': model_name,
            'Test Accuracy': f"{results['test_accuracy']*100:.2f}%",
            'Test Loss': f"{results['test_loss']:.4f}",
            'Top-5 Accuracy': f"{results.get('test_top5_accuracy', 0)*100:.2f}%" if 'test_top5_accuracy' in results else 'N/A'
        })
    
    print("\n📊 Performance Comparison:")
    print("-" * 70)
    for data in comparison_data:
        print(f"\n{data['Model']}:")
        print(f"  Accuracy: {data['Test Accuracy']}")
        print(f"  Loss: {data['Test Loss']}")
        print(f"  Top-5 Acc: {data['Top-5 Accuracy']}")
    
    # Save comparison
    with open('model_comparison.json', 'w') as f:
        json.dump(comparison_data, f, indent=2)
    
    print("\n✓ Comparison saved to: model_comparison.json")


def main():
    """Main execution function"""
    print("\n" + "="*70)
    print("ADVANCED DEFECT DETECTION PIPELINE")
    print("="*70)
    
    DATASET_PATH = "final_merged_dataset"
    EPOCHS = 50
    BATCH_SIZE = 32
    
    results = {}
    
    # 1. EfficientNet
    print("\n\n" + "🔷"*35)
    print("TRAINING EFFICIENTNET")
    print("🔷"*35)
    
    efficientnet = EfficientNetDefectDetector(DATASET_PATH)
    efficientnet.prepare_data(batch_size=BATCH_SIZE)
    efficientnet.build_model(learning_rate=0.001)
    efficientnet.train(epochs=EPOCHS)
    results['EfficientNet'] = efficientnet.evaluate()
    efficientnet.plot_training_history()
    
    # 2. ResNet-50
    print("\n\n" + "🔶"*35)
    print("TRAINING RESNET-50")
    print("🔶"*35)
    
    resnet = ResNet50DefectDetector(DATASET_PATH)
    resnet.prepare_data(batch_size=BATCH_SIZE)
    resnet.build_model(learning_rate=0.001)
    resnet.train(epochs=EPOCHS)
    results['ResNet50'] = resnet.evaluate()
    resnet.plot_training_history()
    
    # 3. Vision Transformer
    print("\n\n" + "🔹"*35)
    print("TRAINING VISION TRANSFORMER")
    print("🔹"*35)
    

    
    # Compare all models
    compare_models(results)
    
    print("\n" + "="*70)
    print("🎉 ALL MODELS TRAINED SUCCESSFULLY!")
    print("="*70)
    print("\n📁 Output directories:")
    print("  - EfficientNetB0_output/")
    print("  - ResNet50_output/")
    
    print("\n✓ Check model_comparison.json for detailed comparison")


if __name__ == "__main__":
    main()


ADVANCED DEFECT DETECTION PIPELINE


🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷
TRAINING EFFICIENTNET
🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

DATA PREPARATION - EfficientNetB0

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING EFFICIENTNET MODEL

✓ Model built successfully!
✓ Total parameters: 4,858,082
✓ Base model frozen: True

TRAINING EfficientNetB0

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-12-07 11:12:35

Epoch 1/50
215/215 [==============================] - 102s 441ms/step - loss: 1.5455 - accuracy: 0.6248 - top_5_accuracy: 0.7945 - val_loss: 1.2169 - val_accuracy: 0.6787 - val_top_5_accuracy: 0.8695 - lr: 0.0010
Epoch 2/50
215/215 [==============================] - 68s 316ms/step - loss: 1.1926 - ac

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import pandas as pd

# ============================================================================
# PART 1: VISION TRANSFORMER TRAINING
# ============================================================================

class VisionTransformerDefectDetector:
    """Vision Transformer (ViT) for defect detection"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224, patch_size=16):
        self.dataset_path = Path(dataset_path)
        self.model_name = "VisionTransformer"
        self.img_height = img_height
        self.img_width = img_width
        self.patch_size = patch_size
        self.num_patches = (img_height // patch_size) * (img_width // patch_size)
        self.model = None
        self.history = None
        self.class_names = []
        self.num_classes = 0
        
        # Create output directories
        self.output_dir = Path("VisionTransformer_output")
        self.model_dir = self.output_dir / "models"
        self.plots_dir = self.output_dir / "plots"
        self.logs_dir = self.output_dir / "logs"
        
        for dir_path in [self.output_dir, self.model_dir, self.plots_dir, self.logs_dir]:
            dir_path.mkdir(exist_ok=True)
    
    def prepare_data(self, validation_split=0.2, test_split=0.1, batch_size=32):
        """Prepare dataset"""
        print("\n" + "="*70)
        print(f"DATA PREPARATION - {self.model_name}")
        print("="*70)
        
        # Get all image paths and labels
        image_paths = []
        labels = []
        
        print("\n📁 Scanning dataset...")
        for class_dir in self.dataset_path.rglob("*"):
            if class_dir.is_dir():
                images = (list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + 
                         list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp")))
                
                if images:
                    class_name = class_dir.name
                    if class_name not in self.class_names:
                        self.class_names.append(class_name)
                    
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_name)
        
        self.class_names.sort()
        self.num_classes = len(self.class_names)
        print(f"✓ Found {len(image_paths)} images")
        print(f"✓ Number of classes: {self.num_classes}")
        
        # Create label mapping
        label_to_index = {label: idx for idx, label in enumerate(self.class_names)}
        labels_encoded = [label_to_index[label] for label in labels]
        
        # Create tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels_encoded))
        dataset = dataset.shuffle(len(image_paths), seed=42)
        
        # Calculate splits
        train_size = int(len(image_paths) * (1 - validation_split - test_split))
        val_size = int(len(image_paths) * validation_split)
        test_size = len(image_paths) - train_size - val_size
        
        # Split datasets
        train_ds = dataset.take(train_size)
        val_ds = dataset.skip(train_size).take(val_size)
        test_ds = dataset.skip(train_size + val_size)
        
        # Process datasets
        self.train_dataset = train_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=True),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.val_dataset = val_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.test_dataset = test_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        print(f"\n✓ Train samples: {train_size}")
        print(f"✓ Validation samples: {val_size}")
        print(f"✓ Test samples: {test_size}")
        
        return train_size, val_size, test_size
    
    def _load_and_preprocess(self, image_path, label, augment=False):
        """Load and preprocess image"""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, [self.img_height, self.img_width])
        
        # Data augmentation
        if augment:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.rot90(img, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
        
        img = tf.clip_by_value(img, 0.0, 255.0)
        img = img / 255.0  # Normalize to [0, 1]
        
        label_onehot = tf.one_hot(label, self.num_classes)
        return img, label_onehot
    
    def build_model(self, transformer_layers=8, num_heads=8, mlp_dim=2048, 
                    dropout_rate=0.1, learning_rate=0.001):
        """Build Vision Transformer model"""
        print("\n" + "="*70)
        print("BUILDING VISION TRANSFORMER MODEL")
        print("="*70)
        
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # Create patches
        patches = self._create_patches(inputs)
        
        # Patch embedding
        projection_dim = 768
        patch_embeddings = layers.Dense(projection_dim)(patches)
        
        # Position embedding
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        position_embeddings = layers.Embedding(
            input_dim=self.num_patches, output_dim=projection_dim
        )(positions)
        
        # Add position embeddings
        encoded = patch_embeddings + position_embeddings
        
        # Transformer blocks
        for _ in range(transformer_layers):
            # Layer normalization 1
            x1 = layers.LayerNormalization(epsilon=1e-6)(encoded)
            
            # Multi-head attention
            attention_output = layers.MultiHeadAttention(
                num_heads=num_heads, key_dim=projection_dim // num_heads, dropout=dropout_rate
            )(x1, x1)
            
            # Skip connection 1
            x2 = layers.Add()([attention_output, encoded])
            
            # Layer normalization 2
            x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
            
            # MLP
            x3 = layers.Dense(mlp_dim, activation='gelu')(x3)
            x3 = layers.Dropout(dropout_rate)(x3)
            x3 = layers.Dense(projection_dim)(x3)
            x3 = layers.Dropout(dropout_rate)(x3)
            
            # Skip connection 2
            encoded = layers.Add()([x3, x2])
        
        # Final layer normalization
        representation = layers.LayerNormalization(epsilon=1e-6)(encoded)
        
        # Global average pooling
        representation = layers.GlobalAveragePooling1D()(representation)
        
        # Classification head
        representation = layers.Dropout(dropout_rate)(representation)
        features = layers.Dense(mlp_dim, activation='gelu')(representation)
        features = layers.Dropout(dropout_rate)(features)
        outputs = layers.Dense(self.num_classes, activation='softmax')(features)
        
        # Create model
        model = keras.Model(inputs=inputs, outputs=outputs)
        
        # Compile - FIXED: Use Adam instead of AdamW
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Vision Transformer built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Number of patches: {self.num_patches}")
        print(f"✓ Transformer layers: {transformer_layers}")
        print(f"✓ Attention heads: {num_heads}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def _create_patches(self, images):
        """Extract patches from images"""
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
        return patches
    
    def train(self, epochs=50, early_stopping_patience=10):
        """Train the model"""
        print("\n" + "="*70)
        print(f"TRAINING {self.model_name}")
        print("="*70)
        
        if self.model is None:
            raise ValueError("Model not built. Call build_model() first.")
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            keras.callbacks.CSVLogger(
                str(self.logs_dir / 'training_log.csv')
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Train model
        self.history = self.model.fit(
            self.train_dataset,
            epochs=epochs,
            validation_data=self.val_dataset,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"\n✓ Training completed!")
        print(f"⏱️  End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save model weights
        self.model.save_weights(str(self.model_dir / 'model_weights.h5'))
        print(f"✓ Model weights saved to: {self.model_dir / 'model_weights.h5'}")
        
        return self.history
    
    def evaluate(self):
        """Evaluate model on test set"""
        print("\n" + "="*70)
        print(f"EVALUATING {self.model_name}")
        print("="*70)
        
        print("\n📊 Evaluating on test set...")
        test_results = self.model.evaluate(self.test_dataset, verbose=1)
        
        print("\n✓ Test Results:")
        print(f"  Loss: {test_results[0]:.4f}")
        print(f"  Accuracy: {test_results[1]:.4f}")
        print(f"  Top-5 Accuracy: {test_results[2]:.4f}")
        
        # Get predictions for confusion matrix
        print("\n📊 Generating detailed metrics...")
        y_true = []
        y_pred = []
        
        for images, labels in self.test_dataset:
            predictions = self.model.predict(images, verbose=0)
            y_true.extend(np.argmax(labels.numpy(), axis=1))
            y_pred.extend(np.argmax(predictions, axis=1))
        
        # Save results
        results = {
            'test_loss': float(test_results[0]),
            'test_accuracy': float(test_results[1]),
            'test_top5_accuracy': float(test_results[2]),
            'num_classes': self.num_classes,
            'evaluation_time': datetime.now().isoformat()
        }
        
        with open(self.logs_dir / 'test_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        # Generate confusion matrix
        self._plot_confusion_matrix(y_true, y_pred)
        
        # Generate classification report
        self._save_classification_report(y_true, y_pred)
        
        return results
    
    def _plot_confusion_matrix(self, y_true, y_pred):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(max(10, self.num_classes), max(8, self.num_classes * 0.8)))
        
        # Normalize confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        sns.heatmap(cm_normalized, annot=False, cmap='Blues', 
                    xticklabels=self.class_names if self.num_classes <= 20 else False,
                    yticklabels=self.class_names if self.num_classes <= 20 else False)
        
        plt.title(f'{self.model_name} - Confusion Matrix (Normalized)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Confusion matrix saved to: {self.plots_dir / 'confusion_matrix.png'}")
    
    def _save_classification_report(self, y_true, y_pred):
        """Save classification report"""
        report = classification_report(y_true, y_pred, target_names=self.class_names, 
                                      zero_division=0)
        
        with open(self.logs_dir / 'classification_report.txt', 'w') as f:
            f.write(f"{self.model_name} - Classification Report\n")
            f.write("="*70 + "\n\n")
            f.write(report)
        
        print(f"✓ Classification report saved to: {self.logs_dir / 'classification_report.txt'}")
    
    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            print("⚠️  No training history available")
            return
        
        print("\n📈 Generating training plots...")
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot accuracy
        axes[0].plot(self.history.history['accuracy'], label='Train Accuracy', linewidth=2)
        axes[0].plot(self.history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
        axes[0].set_title(f'{self.model_name} - Model Accuracy', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Plot loss
        axes[1].plot(self.history.history['loss'], label='Train Loss', linewidth=2)
        axes[1].plot(self.history.history['val_loss'], label='Val Loss', linewidth=2)
        axes[1].set_title(f'{self.model_name} - Model Loss', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'training_history.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Training plots saved to: {self.plots_dir / 'training_history.png'}")



In [3]:
class ModelComparison:
    """Compare multiple trained models"""
    
    def __init__(self, model_dirs):
        """
        Args:
            model_dirs: List of model output directories
        """
        self.model_dirs = [Path(d) for d in model_dirs]
        self.results = {}
        self.load_results()
    
    def load_results(self):
        """Load results from all models"""
        print("\n" + "="*70)
        print("LOADING MODEL RESULTS")
        print("="*70)
        
        for model_dir in self.model_dirs:
            results_file = model_dir / "logs" / "test_results.json"
            
            if results_file.exists():
                with open(results_file, 'r') as f:
                    data = json.load(f)
                    model_name = model_dir.name.replace("_output", "")
                    self.results[model_name] = data
                    print(f"✓ Loaded results for {model_name}")
            else:
                print(f"⚠️  Results not found for {model_dir}")
    
    def generate_comparison_table(self):
        """Generate comparison table"""
        print("\n" + "="*70)
        print("MODEL PERFORMANCE COMPARISON")
        print("="*70)
        
        comparison_data = []
        
        for model_name, results in self.results.items():
            comparison_data.append({
                'Model': model_name,
                'Test Accuracy (%)': f"{results['test_accuracy']*100:.2f}",
                'Test Loss': f"{results['test_loss']:.4f}",
                'Top-5 Accuracy (%)': f"{results.get('test_top5_accuracy', 0)*100:.2f}",
                'Number of Classes': results['num_classes']
            })
        
        # Create DataFrame
        df = pd.DataFrame(comparison_data)
        
        # Sort by accuracy
        df['Accuracy_Numeric'] = df['Test Accuracy (%)'].astype(float)
        df = df.sort_values('Accuracy_Numeric', ascending=False)
        df = df.drop('Accuracy_Numeric', axis=1)
        
        print("\n" + "="*70)
        print(df.to_string(index=False))
        print("="*70)
        
        # Save to CSV
        df.to_csv('model_comparison.csv', index=False)
        print(f"\n✓ Comparison table saved to: model_comparison.csv")
        
        # Save to JSON
        with open('model_comparison.json', 'w') as f:
            json.dump(comparison_data, f, indent=2)
        print(f"✓ Comparison data saved to: model_comparison.json")
        
        return df
    
    def plot_comparison_charts(self):
        """Generate comparison visualizations"""
        print("\n📊 Generating comparison charts...")
        
        if len(self.results) == 0:
            print("⚠️  No results to compare")
            return
        
        models = list(self.results.keys())
        accuracies = [self.results[m]['test_accuracy'] * 100 for m in models]
        losses = [self.results[m]['test_loss'] for m in models]
        top5_accs = [self.results[m].get('test_top5_accuracy', 0) * 100 for m in models]
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        
        # Color scheme
        colors = ['#3498db', '#e74c3c', '#2ecc71']
        
        # 1. Test Accuracy Comparison
        bars1 = axes[0].bar(models, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
        axes[0].set_title('Test Accuracy Comparison', fontsize=14, fontweight='bold')
        axes[0].set_ylabel('Accuracy (%)', fontsize=12)
        axes[0].set_ylim([0, 100])
        axes[0].grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for bar, acc in zip(bars1, accuracies):
            height = bar.get_height()
            axes[0].text(bar.get_x() + bar.get_width()/2., height,
                        f'{acc:.2f}%', ha='center', va='bottom', fontweight='bold')
        
        # 2. Test Loss Comparison
        bars2 = axes[1].bar(models, losses, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
        axes[1].set_title('Test Loss Comparison', fontsize=14, fontweight='bold')
        axes[1].set_ylabel('Loss', fontsize=12)
        axes[1].grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for bar, loss in zip(bars2, losses):
            height = bar.get_height()
            axes[1].text(bar.get_x() + bar.get_width()/2., height,
                        f'{loss:.4f}', ha='center', va='bottom', fontweight='bold')
        
        # 3. Top-5 Accuracy Comparison
        bars3 = axes[2].bar(models, top5_accs, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
        axes[2].set_title('Top-5 Accuracy Comparison', fontsize=14, fontweight='bold')
        axes[2].set_ylabel('Top-5 Accuracy (%)', fontsize=12)
        axes[2].set_ylim([0, 100])
        axes[2].grid(axis='y', alpha=0.3)
        
        # Add value labels on bars
        for bar, acc in zip(bars3, top5_accs):
            height = bar.get_height()
            axes[2].text(bar.get_x() + bar.get_width()/2., height,
                        f'{acc:.2f}%', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig('model_comparison_charts.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Comparison charts saved to: model_comparison_charts.png")
    
    def generate_summary_report(self):
        """Generate comprehensive summary report"""
        print("\n📝 Generating summary report...")
        
        report_file = 'model_comparison_report.txt'
        
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write("DEFECT DETECTION MODEL COMPARISON REPORT\n")
            f.write("="*80 + "\n\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            # Individual model results
            f.write("-"*80 + "\n")
            f.write("INDIVIDUAL MODEL RESULTS\n")
            f.write("-"*80 + "\n\n")
            
            for model_name, results in self.results.items():
                f.write(f"{model_name}:\n")
                f.write(f"  Test Accuracy: {results['test_accuracy']*100:.2f}%\n")
                f.write(f"  Test Loss: {results['test_loss']:.4f}\n")
                f.write(f"  Top-5 Accuracy: {results.get('test_top5_accuracy', 0)*100:.2f}%\n")
                f.write(f"  Number of Classes: {results['num_classes']}\n\n")
            
            # Best model
            f.write("-"*80 + "\n")
            f.write("BEST PERFORMING MODEL\n")
            f.write("-"*80 + "\n\n")
            
            best_model = max(self.results.items(), key=lambda x: x[1]['test_accuracy'])
            f.write(f"🏆 Winner: {best_model[0]}\n")
            f.write(f"   Accuracy: {best_model[1]['test_accuracy']*100:.2f}%\n")
            f.write(f"   Loss: {best_model[1]['test_loss']:.4f}\n\n")
            
            # Recommendations
            f.write("-"*80 + "\n")
            f.write("RECOMMENDATIONS\n")
            f.write("-"*80 + "\n\n")
            
            accuracies = {name: results['test_accuracy'] for name, results in self.results.items()}
            
            if max(accuracies.values()) < 0.80:
                f.write("⚠️  All models show accuracy < 80%\n")
                f.write("   Consider:\n")
                f.write("   - Training for more epochs\n")
                f.write("   - Fine-tuning pre-trained layers\n")
                f.write("   - Increasing model complexity\n")
                f.write("   - Data augmentation strategies\n\n")
            
            if max(accuracies.values()) >= 0.80:
                f.write("✓ Good performance achieved!\n")
                f.write(f"  Best model ({best_model[0]}) is recommended for deployment.\n\n")
            
            # Model strengths
            f.write("-"*80 + "\n")
            f.write("MODEL STRENGTHS\n")
            f.write("-"*80 + "\n\n")
            
            if 'EfficientNetB0' in self.results:
                f.write("EfficientNetB0:\n")
                f.write("  + Fast inference speed\n")
                f.write("  + Good balance of accuracy and efficiency\n")
                f.write("  + Suitable for production deployment\n\n")
            
            if 'ResNet50' in self.results:
                f.write("ResNet50:\n")
                f.write("  + Robust and well-tested architecture\n")
                f.write("  + Good feature extraction\n")
                f.write("  + Reliable baseline performance\n\n")
            
            if 'VisionTransformer' in self.results:
                f.write("VisionTransformer:\n")
                f.write("  + State-of-the-art architecture\n")
                f.write("  + Captures global context well\n")
                f.write("  + Best for large datasets\n\n")
            
            f.write("="*80 + "\n")
        
        print(f"✓ Summary report saved to: {report_file}")




In [4]:
def train_vision_transformer():
    """Train Vision Transformer model"""
    print("\n\n" + "🔹"*35)
    print("TRAINING VISION TRANSFORMER")
    print("🔹"*35)
    
    DATASET_PATH = "final_merged_dataset"
    EPOCHS = 50
    BATCH_SIZE = 32
    
    vit = VisionTransformerDefectDetector(DATASET_PATH)
    vit.prepare_data(batch_size=BATCH_SIZE)
    vit.build_model(transformer_layers=8, num_heads=8, learning_rate=0.001)
    vit.train(epochs=EPOCHS)
    results = vit.evaluate()
    vit.plot_training_history()
    
    return results


def compare_all_models():
    """Compare all trained models"""
    print("\n\n" + "📊"*35)
    print("COMPARING ALL MODELS")
    print("📊"*35)
    
    # Model directories
    model_dirs = [
        "EfficientNetB0_output",
        "ResNet50_output",
        "VisionTransformer_output"
    ]
    
    # Initialize comparison
    comparison = ModelComparison(model_dirs)
    
    # Generate comparison table
    df = comparison.generate_comparison_table()
    
    # Generate comparison charts
    comparison.plot_comparison_charts()
    
    # Generate summary report
    comparison.generate_summary_report()
    
    print("\n" + "="*70)
    print("🎉 COMPARISON COMPLETED!")
    print("="*70)
    print("\n📁 Output files:")
    print("  - model_comparison.csv")
    print("  - model_comparison.json")
    print("  - model_comparison_charts.png")
    print("  - model_comparison_report.txt")


def main():
    """Main execution"""
    import sys
    
    if len(sys.argv) > 1:
        if sys.argv[1] == "train_vit":
            # Only train Vision Transformer
            train_vision_transformer()
        elif sys.argv[1] == "compare":
            # Only do comparison
            compare_all_models()
        else:
            print("Usage:")
            print("  python script.py train_vit   - Train Vision Transformer only")
            print("  python script.py compare     - Compare all trained models")
            print("  python script.py             - Train ViT and compare all")
    else:
        # Default: Train ViT and then compare all models
        print("\n" + "="*70)
        print("COMPLETE PIPELINE: TRAIN VIT + COMPARE ALL MODELS")
        print("="*70)
        
        # Train Vision Transformer
        train_vision_transformer()
        
        # Compare all models
        compare_all_models()
        
        print("\n" + "="*70)
        print("🎉 ALL TASKS COMPLETED SUCCESSFULLY!")
        print("="*70)


if __name__ == "__main__":
    main()

Usage:
  python script.py train_vit   - Train Vision Transformer only
  python script.py compare     - Compare all trained models
  python script.py             - Train ViT and compare all


In [5]:
train_vision_transformer()



🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
TRAINING VISION TRANSFORMER
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹

DATA PREPARATION - VisionTransformer

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING VISION TRANSFORMER MODEL

✓ Vision Transformer built successfully!
✓ Total parameters: 46,407,999
✓ Number of patches: 196
✓ Transformer layers: 8
✓ Attention heads: 8

TRAINING VisionTransformer

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-11-14 19:08:57

Epoch 1/50
215/215 [==============================] - 90s 400ms/step - loss: 2.4519 - accuracy: 0.4111 - top_5_accuracy: 0.6736 - val_loss: 2.0448 - val_accuracy: 0.4595 - val_top_5_accuracy: 0.7251 - lr: 0.0010
Epoch 2/50
215/215 [==============================] - 86s 399ms/step - loss: 1.9515 - accuracy: 0.5052 - top_5_accuracy: 0.7409 - val_loss: 1.7426 - val_accuracy: 0.5502 - val_top_5_accuracy: 0.7680 - lr: 0.0010
Epoch 3/50
215/215

{'test_loss': 1.5360524654388428,
 'test_accuracy': 0.5743380784988403,
 'test_top5_accuracy': 0.8095722794532776,
 'num_classes': 63,
 'evaluation_time': '2025-11-14T20:21:29.184857'}

In [6]:
compare_all_models()



📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊
COMPARING ALL MODELS
📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊

LOADING MODEL RESULTS
✓ Loaded results for EfficientNetB0
✓ Loaded results for ResNet50
✓ Loaded results for VisionTransformer

MODEL PERFORMANCE COMPARISON

            Model Test Accuracy (%) Test Loss Top-5 Accuracy (%)  Number of Classes
   EfficientNetB0             83.71    0.4259              96.64                 63
         ResNet50             80.35    0.4938              97.15                 63
VisionTransformer             57.43    1.5361              80.96                 63

✓ Comparison table saved to: model_comparison.csv
✓ Comparison data saved to: model_comparison.json

📊 Generating comparison charts...
✓ Comparison charts saved to: model_comparison_charts.png

📝 Generating summary report...
✓ Summary report saved to: model_comparison_report.txt

🎉 COMPARISON COMPLETED!

📁 Output files:
  - model_comparison.csv
  - model_comparison.json
  - model_comparison_charts.png
  - 

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3, VGG16, VGG19
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

class DefectDetectionModel:
    """Base class for defect detection models"""
    
    def __init__(self, dataset_path, model_name, img_height=224, img_width=224):
        self.dataset_path = Path(dataset_path)
        self.model_name = model_name
        self.img_height = img_height
        self.img_width = img_width
        self.model = None
        self.history = None
        self.class_names = []
        self.num_classes = 0
        
        # Create output directories
        self.output_dir = Path(f"{model_name}_output")
        self.model_dir = self.output_dir / "models"
        self.plots_dir = self.output_dir / "plots"
        self.logs_dir = self.output_dir / "logs"
        
        for dir_path in [self.output_dir, self.model_dir, self.plots_dir, self.logs_dir]:
            dir_path.mkdir(exist_ok=True)
    
    def prepare_data(self, validation_split=0.2, test_split=0.1, batch_size=32):
        """Prepare dataset with efficient TensorFlow pipeline"""
        print("\n" + "="*70)
        print(f"DATA PREPARATION - {self.model_name}")
        print("="*70)
        
        # Get all image paths and labels
        image_paths = []
        labels = []
        
        print("\n📁 Scanning dataset...")
        for class_dir in self.dataset_path.rglob("*"):
            if class_dir.is_dir():
                images = (list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + 
                         list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp")))
                
                if images:
                    class_name = class_dir.name
                    if class_name not in self.class_names:
                        self.class_names.append(class_name)
                    
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_name)
        
        self.class_names.sort()
        self.num_classes = len(self.class_names)
        print(f"✓ Found {len(image_paths)} images")
        print(f"✓ Number of classes: {self.num_classes}")
        print(f"✓ Classes: {self.class_names[:10]}{'...' if len(self.class_names) > 10 else ''}")
        
        # Create label mapping
        label_to_index = {label: idx for idx, label in enumerate(self.class_names)}
        labels_encoded = [label_to_index[label] for label in labels]
        
        # Create tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels_encoded))
        dataset = dataset.shuffle(len(image_paths), seed=42)
        
        # Calculate splits
        train_size = int(len(image_paths) * (1 - validation_split - test_split))
        val_size = int(len(image_paths) * validation_split)
        test_size = len(image_paths) - train_size - val_size
        
        # Split datasets
        train_ds = dataset.take(train_size)
        val_ds = dataset.skip(train_size).take(val_size)
        test_ds = dataset.skip(train_size + val_size)
        
        # Process datasets
        self.train_dataset = train_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=True),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.val_dataset = val_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.test_dataset = test_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        print(f"\n✓ Train samples: {train_size}")
        print(f"✓ Validation samples: {val_size}")
        print(f"✓ Test samples: {test_size}")
        
        return train_size, val_size, test_size
    
    def _load_and_preprocess(self, image_path, label, augment=False):
        """Load and preprocess image"""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, [self.img_height, self.img_width])
        
        # Data augmentation
        if augment:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.rot90(img, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
        
        img = tf.clip_by_value(img, 0.0, 255.0)
        img = img / 255.0  # Normalize to [0, 1]
        
        label_onehot = tf.one_hot(label, self.num_classes)
        return img, label_onehot
    
    def train(self, epochs=50, early_stopping_patience=10):
        """Train the model"""
        print("\n" + "="*70)
        print(f"TRAINING {self.model_name}")
        print("="*70)
        
        if self.model is None:
            raise ValueError("Model not built. Call build_model() first.")
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            keras.callbacks.CSVLogger(
                str(self.logs_dir / 'training_log.csv')
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Train model
        self.history = self.model.fit(
            self.train_dataset,
            epochs=epochs,
            validation_data=self.val_dataset,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"\n✓ Training completed!")
        print(f"⏱️  End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save model weights
        self.model.save_weights(str(self.model_dir / 'model_weights.h5'))
        print(f"✓ Model weights saved to: {self.model_dir / 'model_weights.h5'}")
        
        return self.history
    
    def evaluate(self):
        """Evaluate model on test set"""
        print("\n" + "="*70)
        print(f"EVALUATING {self.model_name}")
        print("="*70)
        
        print("\n📊 Evaluating on test set...")
        test_results = self.model.evaluate(self.test_dataset, verbose=1)
        
        print("\n✓ Test Results:")
        print(f"  Loss: {test_results[0]:.4f}")
        print(f"  Accuracy: {test_results[1]:.4f}")
        if len(test_results) > 2:
            print(f"  Top-5 Accuracy: {test_results[2]:.4f}")
        
        # Get predictions for confusion matrix
        print("\n📊 Generating detailed metrics...")
        y_true = []
        y_pred = []
        
        for images, labels in self.test_dataset:
            predictions = self.model.predict(images, verbose=0)
            y_true.extend(np.argmax(labels.numpy(), axis=1))
            y_pred.extend(np.argmax(predictions, axis=1))
        
        # Save results
        results = {
            'test_loss': float(test_results[0]),
            'test_accuracy': float(test_results[1]),
            'num_classes': self.num_classes,
            'evaluation_time': datetime.now().isoformat()
        }
        
        if len(test_results) > 2:
            results['test_top5_accuracy'] = float(test_results[2])
        
        with open(self.logs_dir / 'test_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        # Generate confusion matrix
        self._plot_confusion_matrix(y_true, y_pred)
        
        # Generate classification report
        self._save_classification_report(y_true, y_pred)
        
        return results
    
    def _plot_confusion_matrix(self, y_true, y_pred):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(max(10, self.num_classes), max(8, self.num_classes * 0.8)))
        
        # Normalize confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        sns.heatmap(cm_normalized, annot=False, cmap='Blues', 
                    xticklabels=self.class_names if self.num_classes <= 20 else False,
                    yticklabels=self.class_names if self.num_classes <= 20 else False)
        
        plt.title(f'{self.model_name} - Confusion Matrix (Normalized)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Confusion matrix saved to: {self.plots_dir / 'confusion_matrix.png'}")
    
    def _save_classification_report(self, y_true, y_pred):
        """Save classification report"""
        report = classification_report(y_true, y_pred, target_names=self.class_names, 
                                      zero_division=0)
        
        with open(self.logs_dir / 'classification_report.txt', 'w') as f:
            f.write(f"{self.model_name} - Classification Report\n")
            f.write("="*70 + "\n\n")
            f.write(report)
        
        print(f"✓ Classification report saved to: {self.logs_dir / 'classification_report.txt'}")
    
    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            print("⚠️  No training history available")
            return
        
        print("\n📈 Generating training plots...")
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot accuracy
        axes[0].plot(self.history.history['accuracy'], label='Train Accuracy', linewidth=2)
        axes[0].plot(self.history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
        axes[0].set_title(f'{self.model_name} - Model Accuracy', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Plot loss
        axes[1].plot(self.history.history['loss'], label='Train Loss', linewidth=2)
        axes[1].plot(self.history.history['val_loss'], label='Val Loss', linewidth=2)
        axes[1].set_title(f'{self.model_name} - Model Loss', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'training_history.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Training plots saved to: {self.plots_dir / 'training_history.png'}")


class InceptionV3DefectDetector(DefectDetectionModel):
    """InceptionV3 for defect detection"""
    
    def __init__(self, dataset_path, img_height=299, img_width=299):
        # InceptionV3 requires 299x299 input
        super().__init__(dataset_path, "InceptionV3", img_height, img_width)
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.3):
        """Build InceptionV3 model with transfer learning"""
        print("\n" + "="*70)
        print("BUILDING INCEPTIONV3 MODEL")
        print("="*70)
        
        # Load pre-trained InceptionV3
        base_model = InceptionV3(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Freeze base model initially
        base_model.trainable = False
        
        # Build model
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # InceptionV3 preprocessing (scale to [-1, 1])
        x = layers.Rescaling(scale=2.0, offset=-1.0)(inputs)
        
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Base model frozen: {not base_model.trainable}")
        print(f"✓ Input size: {self.img_height}x{self.img_width}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def unfreeze_and_fine_tune(self, fine_tune_from_layer=249, learning_rate=0.0001):
        """Unfreeze layers for fine-tuning"""
        print(f"\n🔓 Unfreezing layers from layer {fine_tune_from_layer}...")
        
        base_model = self.model.layers[2]  # Get InceptionV3 base
        base_model.trainable = True
        
        # Freeze layers before fine_tune_from_layer
        for layer in base_model.layers[:fine_tune_from_layer]:
            layer.trainable = False
        
        # Recompile
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        print(f"✓ Fine-tuning enabled with learning rate: {learning_rate}")


class VGG16DefectDetector(DefectDetectionModel):
    """VGG16 for defect detection"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224):
        super().__init__(dataset_path, "VGG16", img_height, img_width)
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.5):
        """Build VGG16 model with transfer learning"""
        print("\n" + "="*70)
        print("BUILDING VGG16 MODEL")
        print("="*70)
        
        # Load pre-trained VGG16
        base_model = VGG16(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Freeze base model initially
        base_model.trainable = False
        
        # Build model
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # VGG preprocessing (mean subtraction)
        x = tf.keras.applications.vgg16.preprocess_input(inputs * 255.0)
        
        x = base_model(x, training=False)
        x = layers.Flatten()(x)
        x = layers.Dense(4096, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(2048, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Base model frozen: {not base_model.trainable}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def unfreeze_and_fine_tune(self, fine_tune_from_layer=15, learning_rate=0.0001):
        """Unfreeze layers for fine-tuning"""
        print(f"\n🔓 Unfreezing layers from layer {fine_tune_from_layer}...")
        
        base_model = self.model.layers[2]  # Get VGG16 base
        base_model.trainable = True
        
        # Freeze layers before fine_tune_from_layer
        for layer in base_model.layers[:fine_tune_from_layer]:
            layer.trainable = False
        
        # Recompile
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        print(f"✓ Fine-tuning enabled with learning rate: {learning_rate}")


class VGG19DefectDetector(DefectDetectionModel):
    """VGG19 for defect detection"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224):
        super().__init__(dataset_path, "VGG19", img_height, img_width)
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.5):
        """Build VGG19 model with transfer learning"""
        print("\n" + "="*70)
        print("BUILDING VGG19 MODEL")
        print("="*70)
        
        # Load pre-trained VGG19
        base_model = VGG19(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Freeze base model initially
        base_model.trainable = False
        
        # Build model
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # VGG preprocessing (mean subtraction)
        x = tf.keras.applications.vgg19.preprocess_input(inputs * 255.0)
        
        x = base_model(x, training=False)
        x = layers.Flatten()(x)
        x = layers.Dense(4096, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(2048, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Base model frozen: {not base_model.trainable}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def unfreeze_and_fine_tune(self, fine_tune_from_layer=17, learning_rate=0.0001):
        """Unfreeze layers for fine-tuning"""
        print(f"\n🔓 Unfreezing layers from layer {fine_tune_from_layer}...")
        
        base_model = self.model.layers[2]  # Get VGG19 base
        base_model.trainable = True
        
        # Freeze layers before fine_tune_from_layer
        for layer in base_model.layers[:fine_tune_from_layer]:
            layer.trainable = False
        
        # Recompile
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        print(f"✓ Fine-tuning enabled with learning rate: {learning_rate}")


# ============================================================================
# TRAINING FUNCTIONS
# ============================================================================

def train_inceptionv3(dataset_path="final_merged_dataset", epochs=50, batch_size=32):
    """Train InceptionV3 model"""
    print("\n\n" + "🔶"*35)
    print("TRAINING INCEPTIONV3")
    print("🔶"*35)
    
    model = InceptionV3DefectDetector(dataset_path)
    model.prepare_data(batch_size=batch_size)
    model.build_model(learning_rate=0.001)
    model.train(epochs=epochs)
    results = model.evaluate()
    model.plot_training_history()
    
    return results


def train_vgg16(dataset_path="final_merged_dataset", epochs=50, batch_size=32):
    """Train VGG16 model"""
    print("\n\n" + "🔷"*35)
    print("TRAINING VGG16")
    print("🔷"*35)
    
    model = VGG16DefectDetector(dataset_path)
    model.prepare_data(batch_size=batch_size)
    model.build_model(learning_rate=0.001)
    model.train(epochs=epochs)
    results = model.evaluate()
    model.plot_training_history()
    
    return results


def train_vgg19(dataset_path="final_merged_dataset", epochs=50, batch_size=32):
    """Train VGG19 model"""
    print("\n\n" + "🔹"*35)
    print("TRAINING VGG19")
    print("🔹"*35)
    
    model = VGG19DefectDetector(dataset_path)
    model.prepare_data(batch_size=batch_size)
    model.build_model(learning_rate=0.001)
    model.train(epochs=epochs)
    results = model.evaluate()
    model.plot_training_history()
    
    return results


def train_all_three(dataset_path="final_merged_dataset", epochs=30, batch_size=32):
    """Train all three models"""
    print("\n" + "="*70)
    print("TRAINING ALL THREE MODELS: INCEPTIONV3, VGG16, VGG19")
    print("="*70)
    
    results = {}
    
    # Train InceptionV3
    results['InceptionV3'] = train_inceptionv3(dataset_path, epochs, batch_size)
    
    # Train VGG16
    results['VGG16'] = train_vgg16(dataset_path, epochs, batch_size)
    
    # Train VGG19
    results['VGG19'] = train_vgg19(dataset_path, epochs, batch_size)
    
    print("\n" + "="*70)
    print("🎉 ALL THREE MODELS TRAINED SUCCESSFULLY!")
    print("="*70)
    print("\n📁 Output directories:")
    print("  - InceptionV3_output/")
    print("  - VGG16_output/")
    print("  - VGG19_output/")
    
    return results

In [2]:
train_inceptionv3()



🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶
TRAINING INCEPTIONV3
🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶

DATA PREPARATION - InceptionV3

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING INCEPTIONV3 MODEL

✓ Model built successfully!
✓ Total parameters: 24,466,271
✓ Base model frozen: True
✓ Input size: 299x299

TRAINING InceptionV3

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-11-18 20:51:38

Epoch 1/50
215/215 [==============================] - 104s 453ms/step - loss: 1.6010 - accuracy: 0.6226 - top_5_accuracy: 0.8019 - val_loss: 1.2104 - val_accuracy: 0.6757 - val_top_5_accuracy: 0.8781 - lr: 0.0010
Epoch 2/50
215/215 [==============================] - 63s 291ms/step - loss: 1.2210 - accuracy: 0.6593 - top_

{'test_loss': 0.4571196436882019,
 'test_accuracy': 0.8197556138038635,
 'num_classes': 63,
 'evaluation_time': '2025-11-18T21:28:29.666453',
 'test_top5_accuracy': 0.9653767943382263}

In [3]:
train_vgg16()



🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷
TRAINING VGG16
🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

DATA PREPARATION - VGG16

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING VGG16 MODEL

✓ Model built successfully!
✓ Total parameters: 128,032,639
✓ Base model frozen: True

TRAINING VGG16

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-11-18 21:31:20

Epoch 1/50
215/215 [==============================] - 63s 265ms/step - loss: 13.4528 - accuracy: 0.5024 - top_5_accuracy: 0.6494 - val_loss: 2.2588 - val_accuracy: 0.4691 - val_top_5_accuracy: 0.6609 - lr: 0.0010
Epoch 2/50
215/215 [==============================] - 49s 226ms/step - loss: 2.4995 - accuracy: 0.5368 - top_5_accuracy: 0.6938 - val_loss: 1.5817 - val_a

{'test_loss': 1.1026062965393066,
 'test_accuracy': 0.6425662040710449,
 'num_classes': 63,
 'evaluation_time': '2025-11-18T22:03:48.297608',
 'test_top5_accuracy': 0.8462321758270264}

In [2]:
train_vgg19()



🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹
TRAINING VGG19
🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹

DATA PREPARATION - VGG19

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING VGG19 MODEL
80134624/80134624 [==============================] - 9s 0us/step

✓ Model built successfully!
✓ Total parameters: 133,342,335
✓ Base model frozen: True

TRAINING VGG19

🚀 Starting training for 30 epochs...
⏱️  Start time: 2025-11-18 11:36:36

Epoch 1/30
215/215 [==============================] - 117s 502ms/step - loss: 12.7835 - accuracy: 0.5088 - top_5_accuracy: 0.6689 - val_loss: 2.1291 - val_accuracy: 0.6022 - val_top_5_accuracy: 0.7425 - lr: 0.0010
Epoch 2/30
215/215 [==============================] - 68s 315ms/step - loss: 2.5216 - ac

{'test_loss': 1.1955108642578125,
 'test_accuracy': 0.6272912621498108,
 'num_classes': 63,
 'evaluation_time': '2025-11-18T12:06:03.325095',
 'test_top5_accuracy': 0.8380855321884155}

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import Xception
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

class XceptionDefectDetector:
    """Xception for defect detection"""
    
    def __init__(self, dataset_path, img_height=299, img_width=299):
        # Xception requires minimum 71x71, but 299x299 is optimal
        self.dataset_path = Path(dataset_path)
        self.model_name = "Xception"
        self.img_height = img_height
        self.img_width = img_width
        self.model = None
        self.history = None
        self.class_names = []
        self.num_classes = 0
        
        # Create output directories
        self.output_dir = Path("Xception_output")
        self.model_dir = self.output_dir / "models"
        self.plots_dir = self.output_dir / "plots"
        self.logs_dir = self.output_dir / "logs"
        
        for dir_path in [self.output_dir, self.model_dir, self.plots_dir, self.logs_dir]:
            dir_path.mkdir(exist_ok=True)
    
    def prepare_data(self, validation_split=0.2, test_split=0.1, batch_size=32):
        """Prepare dataset with efficient TensorFlow pipeline"""
        print("\n" + "="*70)
        print(f"DATA PREPARATION - {self.model_name}")
        print("="*70)
        
        # Get all image paths and labels
        image_paths = []
        labels = []
        
        print("\n📁 Scanning dataset...")
        for class_dir in self.dataset_path.rglob("*"):
            if class_dir.is_dir():
                images = (list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + 
                         list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp")))
                
                if images:
                    class_name = class_dir.name
                    if class_name not in self.class_names:
                        self.class_names.append(class_name)
                    
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_name)
        
        self.class_names.sort()
        self.num_classes = len(self.class_names)
        print(f"✓ Found {len(image_paths)} images")
        print(f"✓ Number of classes: {self.num_classes}")
        print(f"✓ Classes: {self.class_names[:10]}{'...' if len(self.class_names) > 10 else ''}")
        
        # Create label mapping
        label_to_index = {label: idx for idx, label in enumerate(self.class_names)}
        labels_encoded = [label_to_index[label] for label in labels]
        
        # Create tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels_encoded))
        dataset = dataset.shuffle(len(image_paths), seed=42)
        
        # Calculate splits
        train_size = int(len(image_paths) * (1 - validation_split - test_split))
        val_size = int(len(image_paths) * validation_split)
        test_size = len(image_paths) - train_size - val_size
        
        # Split datasets
        train_ds = dataset.take(train_size)
        val_ds = dataset.skip(train_size).take(val_size)
        test_ds = dataset.skip(train_size + val_size)
        
        # Process datasets
        self.train_dataset = train_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=True),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.val_dataset = val_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.test_dataset = test_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        print(f"\n✓ Train samples: {train_size}")
        print(f"✓ Validation samples: {val_size}")
        print(f"✓ Test samples: {test_size}")
        
        return train_size, val_size, test_size
    
    def _load_and_preprocess(self, image_path, label, augment=False):
        """Load and preprocess image"""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, [self.img_height, self.img_width])
        
        # Data augmentation
        if augment:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.rot90(img, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
        
        img = tf.clip_by_value(img, 0.0, 255.0)
        img = img / 255.0  # Normalize to [0, 1]
        
        label_onehot = tf.one_hot(label, self.num_classes)
        return img, label_onehot
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.3):
        """Build Xception model with transfer learning"""
        print("\n" + "="*70)
        print("BUILDING XCEPTION MODEL")
        print("="*70)
        
        # Load pre-trained Xception
        base_model = Xception(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3)
        )
        
        # Freeze base model initially
        base_model.trainable = False
        
        # Build model
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        
        # Xception preprocessing (scale to [-1, 1])
        x = layers.Rescaling(scale=2.0, offset=-1.0)(inputs)
        
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        
        # Dense layers with dropout
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 4)(x)
        
        # Output layer
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        print("\n✓ Xception model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Trainable parameters: {sum([np.prod(v.shape) for v in model.trainable_weights]):,}")
        print(f"✓ Base model frozen: {not base_model.trainable}")
        print(f"✓ Input size: {self.img_height}x{self.img_width}")
        print("\n📝 Model Architecture Highlights:")
        print("  - Depthwise Separable Convolutions")
        print("  - 36 Convolutional Layers")
        print("  - Efficient parameter usage")
        print("  - Excellent for transfer learning")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def unfreeze_and_fine_tune(self, fine_tune_from_layer=100, learning_rate=0.0001):
        """
        Unfreeze layers for fine-tuning
        
        Args:
            fine_tune_from_layer: Layer index to start unfreezing from
                                  Xception has 132 layers total
                                  Good starting points: 100, 116, 126
            learning_rate: Lower learning rate for fine-tuning
        """
        print(f"\n🔓 Unfreezing layers from layer {fine_tune_from_layer}...")
        
        base_model = self.model.layers[2]  # Get Xception base
        base_model.trainable = True
        
        # Freeze layers before fine_tune_from_layer
        for layer in base_model.layers[:fine_tune_from_layer]:
            layer.trainable = False
        
        # Count trainable parameters
        trainable_params = sum([np.prod(v.shape) for v in self.model.trainable_weights])
        
        # Recompile with lower learning rate
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        print(f"✓ Fine-tuning enabled!")
        print(f"✓ Trainable parameters: {trainable_params:,}")
        print(f"✓ Learning rate: {learning_rate}")
    
    def train(self, epochs=50, early_stopping_patience=10):
        """Train the model"""
        print("\n" + "="*70)
        print(f"TRAINING {self.model_name}")
        print("="*70)
        
        if self.model is None:
            raise ValueError("Model not built. Call build_model() first.")
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            keras.callbacks.CSVLogger(
                str(self.logs_dir / 'training_log.csv')
            ),
            keras.callbacks.TensorBoard(
                log_dir=str(self.logs_dir / 'tensorboard'),
                histogram_freq=1
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Train model
        self.history = self.model.fit(
            self.train_dataset,
            epochs=epochs,
            validation_data=self.val_dataset,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"\n✓ Training completed!")
        print(f"⏱️  End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save model weights
        self.model.save_weights(str(self.model_dir / 'model_weights.h5'))
        print(f"✓ Model weights saved to: {self.model_dir / 'model_weights.h5'}")
        
        # Save training history
        history_dict = {key: [float(val) for val in values] 
                       for key, values in self.history.history.items()}
        with open(self.logs_dir / 'training_history.json', 'w') as f:
            json.dump(history_dict, f, indent=2)
        
        return self.history
    
    def evaluate(self):
        """Evaluate model on test set"""
        print("\n" + "="*70)
        print(f"EVALUATING {self.model_name}")
        print("="*70)
        
        print("\n📊 Evaluating on test set...")
        test_results = self.model.evaluate(self.test_dataset, verbose=1)
        
        print("\n✓ Test Results:")
        print(f"  Loss: {test_results[0]:.4f}")
        print(f"  Accuracy: {test_results[1]:.4f}")
        print(f"  Top-5 Accuracy: {test_results[2]:.4f}")
        
        # Get predictions for confusion matrix
        print("\n📊 Generating detailed metrics...")
        y_true = []
        y_pred = []
        y_pred_probs = []
        
        for images, labels in self.test_dataset:
            predictions = self.model.predict(images, verbose=0)
            y_true.extend(np.argmax(labels.numpy(), axis=1))
            y_pred.extend(np.argmax(predictions, axis=1))
            y_pred_probs.extend(predictions)
        
        # Calculate per-class accuracy
        per_class_accuracy = {}
        for i, class_name in enumerate(self.class_names):
            class_mask = np.array(y_true) == i
            if class_mask.sum() > 0:
                class_acc = (np.array(y_pred)[class_mask] == i).mean()
                per_class_accuracy[class_name] = float(class_acc)
        
        # Save results
        results = {
            'test_loss': float(test_results[0]),
            'test_accuracy': float(test_results[1]),
            'test_top5_accuracy': float(test_results[2]),
            'num_classes': self.num_classes,
            'per_class_accuracy': per_class_accuracy,
            'evaluation_time': datetime.now().isoformat()
        }
        
        with open(self.logs_dir / 'test_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        # Generate confusion matrix
        self._plot_confusion_matrix(y_true, y_pred)
        
        # Generate classification report
        self._save_classification_report(y_true, y_pred)
        
        # Plot top-k accuracy curve
        self._plot_topk_accuracy(y_true, y_pred_probs)
        
        return results
    
    def _plot_confusion_matrix(self, y_true, y_pred):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(max(12, self.num_classes * 0.5), max(10, self.num_classes * 0.4)))
        
        # Normalize confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        sns.heatmap(cm_normalized, annot=False, cmap='Blues', cbar=True,
                    xticklabels=self.class_names if self.num_classes <= 20 else False,
                    yticklabels=self.class_names if self.num_classes <= 20 else False,
                    square=True)
        
        plt.title(f'{self.model_name} - Confusion Matrix (Normalized)', 
                 fontsize=16, fontweight='bold', pad=20)
        plt.ylabel('True Label', fontsize=12)
        plt.xlabel('Predicted Label', fontsize=12)
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Confusion matrix saved to: {self.plots_dir / 'confusion_matrix.png'}")
    
    def _save_classification_report(self, y_true, y_pred):
        """Save classification report"""
        report = classification_report(y_true, y_pred, target_names=self.class_names, 
                                      zero_division=0)
        
        with open(self.logs_dir / 'classification_report.txt', 'w') as f:
            f.write(f"{self.model_name} - Classification Report\n")
            f.write("="*70 + "\n\n")
            f.write(report)
        
        print(f"✓ Classification report saved to: {self.logs_dir / 'classification_report.txt'}")
    
    def _plot_topk_accuracy(self, y_true, y_pred_probs):
        """Plot top-k accuracy curve"""
        y_pred_probs = np.array(y_pred_probs)
        y_true = np.array(y_true)
        
        k_values = range(1, min(11, self.num_classes + 1))
        top_k_accs = []
        
        for k in k_values:
            top_k_preds = np.argsort(y_pred_probs, axis=1)[:, -k:]
            correct = np.array([y_true[i] in top_k_preds[i] for i in range(len(y_true))])
            top_k_accs.append(correct.mean())
        
        plt.figure(figsize=(10, 6))
        plt.plot(k_values, np.array(top_k_accs) * 100, marker='o', linewidth=2, markersize=8)
        plt.xlabel('K', fontsize=12)
        plt.ylabel('Top-K Accuracy (%)', fontsize=12)
        plt.title(f'{self.model_name} - Top-K Accuracy', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'topk_accuracy.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Top-K accuracy plot saved to: {self.plots_dir / 'topk_accuracy.png'}")
    
    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            print("⚠️  No training history available")
            return
        
        print("\n📈 Generating training plots...")
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Plot accuracy
        axes[0, 0].plot(self.history.history['accuracy'], label='Train Accuracy', 
                       linewidth=2, color='#3498db')
        axes[0, 0].plot(self.history.history['val_accuracy'], label='Val Accuracy', 
                       linewidth=2, color='#e74c3c')
        axes[0, 0].set_title(f'{self.model_name} - Accuracy', fontsize=14, fontweight='bold')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Accuracy')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Plot loss
        axes[0, 1].plot(self.history.history['loss'], label='Train Loss', 
                       linewidth=2, color='#3498db')
        axes[0, 1].plot(self.history.history['val_loss'], label='Val Loss', 
                       linewidth=2, color='#e74c3c')
        axes[0, 1].set_title(f'{self.model_name} - Loss', fontsize=14, fontweight='bold')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Loss')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Plot top-5 accuracy
        axes[1, 0].plot(self.history.history['top_5_accuracy'], label='Train Top-5', 
                       linewidth=2, color='#3498db')
        axes[1, 0].plot(self.history.history['val_top_5_accuracy'], label='Val Top-5', 
                       linewidth=2, color='#e74c3c')
        axes[1, 0].set_title(f'{self.model_name} - Top-5 Accuracy', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Top-5 Accuracy')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Plot learning rate if available
        if 'lr' in self.history.history:
            axes[1, 1].plot(self.history.history['lr'], linewidth=2, color='#2ecc71')
            axes[1, 1].set_title(f'{self.model_name} - Learning Rate', fontsize=14, fontweight='bold')
            axes[1, 1].set_xlabel('Epoch')
            axes[1, 1].set_ylabel('Learning Rate')
            axes[1, 1].set_yscale('log')
            axes[1, 1].grid(True, alpha=0.3)
        else:
            # Show summary statistics
            final_acc = self.history.history['val_accuracy'][-1]
            best_acc = max(self.history.history['val_accuracy'])
            axes[1, 1].text(0.5, 0.6, f'Final Val Acc: {final_acc:.4f}', 
                           ha='center', fontsize=14, fontweight='bold')
            axes[1, 1].text(0.5, 0.4, f'Best Val Acc: {best_acc:.4f}', 
                           ha='center', fontsize=14, fontweight='bold')
            axes[1, 1].set_title('Summary', fontsize=14, fontweight='bold')
            axes[1, 1].axis('off')
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'training_history.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Training plots saved to: {self.plots_dir / 'training_history.png'}")


# ============================================================================
# TRAINING FUNCTION
# ============================================================================

def train_xception(dataset_path="final_merged_dataset", epochs=50, batch_size=32):
    """Train Xception model"""
    print("\n\n" + "🔶"*35)
    print("TRAINING XCEPTION")
    print("🔶"*35)
    
    model = XceptionDefectDetector(dataset_path)
    model.prepare_data(batch_size=batch_size)
    model.build_model(learning_rate=0.001, dropout_rate=0.3)
    model.train(epochs=epochs, early_stopping_patience=10)
    results = model.evaluate()
    model.plot_training_history()
    
    print("\n" + "="*70)
    print("🎉 XCEPTION TRAINING COMPLETED!")
    print("="*70)
    print(f"\n📊 Final Results:")
    print(f"  Test Accuracy: {results['test_accuracy']*100:.2f}%")
    print(f"  Test Loss: {results['test_loss']:.4f}")
    print(f"  Top-5 Accuracy: {results['test_top5_accuracy']*100:.2f}%")
    print(f"\n📁 All outputs saved to: Xception_output/")
    
    return results


# For fine-tuning after initial training
def fine_tune_xception(model, epochs=20):
    """
    Fine-tune Xception model by unfreezing layers
    Call this after initial training for better performance
    """
    print("\n" + "="*70)
    print("FINE-TUNING XCEPTION")
    print("="*70)
    
    # Unfreeze from layer 100 (out of 132 total layers)
    model.unfreeze_and_fine_tune(fine_tune_from_layer=100, learning_rate=0.0001)
    
    # Continue training with lower learning rate
    model.train(epochs=epochs, early_stopping_patience=7)
    results = model.evaluate()
    model.plot_training_history()
    
    return results

In [6]:
train_xception()



🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶
TRAINING XCEPTION
🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶🔶

DATA PREPARATION - Xception

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING XCEPTION MODEL

✓ Xception model built successfully!
✓ Total parameters: 23,640,167
✓ Trainable parameters: 2,774,591
✓ Base model frozen: True
✓ Input size: 299x299

📝 Model Architecture Highlights:
  - Depthwise Separable Convolutions
  - 36 Convolutional Layers
  - Efficient parameter usage
  - Excellent for transfer learning

TRAINING Xception

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-11-18 13:02:55

Epoch 1/50
215/215 [==============================] - 59s 266ms/step - loss: 1.6032 - accuracy: 0.6242 - top_5_accuracy: 0.789

{'test_loss': 0.49756142497062683,
 'test_accuracy': 0.8197556138038635,
 'test_top5_accuracy': 0.9704684615135193,
 'num_classes': 63,
 'per_class_accuracy': {'Missing_hole': 0.0,
  'Missing_hole_rotation': 0.3333333333333333,
  'Mouse_bite': 0.0,
  'Mouse_bite_rotation': 0.0,
  'Open_circuit': 0.26666666666666666,
  'Open_circuit_rotation': 0.0,
  'PCB_USED': 0.0,
  'Short': 0.8461538461538461,
  'Short_rotation': 0.5,
  'Spur': 0.0,
  'Spur_rotation': 0.0,
  'Spurious_copper': 0.08333333333333333,
  'Spurious_copper_rotation': 0.07692307692307693,
  'bent': 0.0,
  'bent_lead': 1.0,
  'bent_wire': 1.0,
  'broken': 0.5,
  'broken_large': 1.0,
  'broken_small': 0.75,
  'broken_teeth': 0.5,
  'cable_swap': 0.75,
  'color': 0.5833333333333334,
  'combined': 0.625,
  'contamination': 0.8,
  'crack': 0.5714285714285714,
  'cut': 1.0,
  'cut_inner_insulation': 1.0,
  'cut_lead': 1.0,
  'cut_outer_insulation': 1.0,
  'damaged_case': 0.5,
  'defective': 1.0,
  'fabric_border': 1.0,
  'fabric_

In [4]:
"""
Pre-trained Vision Transformer for Defect Detection
Model: ViT-B16 (Vision Transformer Base with 16x16 patches)
Source: keras-cv or vit-keras library
"""

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Install required library (run this first in your notebook)
# !pip install vit-keras

from vit_keras import vit


class PretrainedViTDefectDetector:
    """
    Pre-trained Vision Transformer for defect detection
    Model: ViT-B16 (Base model with 16x16 patches)
    Pre-trained on ImageNet-21k
    """
    
    def __init__(self, dataset_path, img_height=224, img_width=224):
        self.dataset_path = Path(dataset_path)
        self.model_name = "ViT-B16-Pretrained"
        self.img_height = img_height
        self.img_width = img_width
        self.model = None
        self.history = None
        self.class_names = []
        self.num_classes = 0
        
        # Create output directories
        self.output_dir = Path("ViT_B16_Pretrained_output")
        self.model_dir = self.output_dir / "models"
        self.plots_dir = self.output_dir / "plots"
        self.logs_dir = self.output_dir / "logs"
        
        for dir_path in [self.output_dir, self.model_dir, self.plots_dir, self.logs_dir]:
            dir_path.mkdir(exist_ok=True)
    
    def prepare_data(self, validation_split=0.2, test_split=0.1, batch_size=32):
        """Prepare dataset"""
        print("\n" + "="*70)
        print(f"DATA PREPARATION - {self.model_name}")
        print("="*70)
        
        # Get all image paths and labels
        image_paths = []
        labels = []
        
        print("\n📁 Scanning dataset...")
        for class_dir in self.dataset_path.rglob("*"):
            if class_dir.is_dir():
                images = (list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + 
                         list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp")))
                
                if images:
                    class_name = class_dir.name
                    if class_name not in self.class_names:
                        self.class_names.append(class_name)
                    
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_name)
        
        self.class_names.sort()
        self.num_classes = len(self.class_names)
        print(f"✓ Found {len(image_paths)} images")
        print(f"✓ Number of classes: {self.num_classes}")
        
        # Create label mapping
        label_to_index = {label: idx for idx, label in enumerate(self.class_names)}
        labels_encoded = [label_to_index[label] for label in labels]
        
        # Create tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels_encoded))
        dataset = dataset.shuffle(len(image_paths), seed=42)
        
        # Calculate splits
        train_size = int(len(image_paths) * (1 - validation_split - test_split))
        val_size = int(len(image_paths) * validation_split)
        test_size = len(image_paths) - train_size - val_size
        
        # Split datasets
        train_ds = dataset.take(train_size)
        val_ds = dataset.skip(train_size).take(val_size)
        test_ds = dataset.skip(train_size + val_size)
        
        # Process datasets
        self.train_dataset = train_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=True),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.val_dataset = val_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.test_dataset = test_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        print(f"\n✓ Train samples: {train_size}")
        print(f"✓ Validation samples: {val_size}")
        print(f"✓ Test samples: {test_size}")
        
        return train_size, val_size, test_size
    
    def _load_and_preprocess(self, image_path, label, augment=False):
        """Load and preprocess image"""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, [self.img_height, self.img_width])
        
        # Data augmentation
        if augment:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.rot90(img, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
        
        img = tf.clip_by_value(img, 0.0, 255.0)
        
        # ViT expects images in [0, 1] range
        img = img / 255.0
        
        label_onehot = tf.one_hot(label, self.num_classes)
        return img, label_onehot
    
    def build_model(self, model_type='B16', learning_rate=0.001, dropout_rate=0.1):
        """
        Build pre-trained ViT model
        
        Args:
            model_type: 'B16', 'B32', 'L16', 'L32'
                B = Base (12 layers, 768 hidden, 12 heads) - ~86M params
                L = Large (24 layers, 1024 hidden, 16 heads) - ~307M params
                16/32 = patch size
            learning_rate: Initial learning rate
            dropout_rate: Dropout rate for classification head
        """
        print("\n" + "="*70)
        print(f"BUILDING PRE-TRAINED VISION TRANSFORMER (ViT-{model_type})")
        print("="*70)
        
        # Load pre-trained ViT model
        print("\n⏳ Downloading pre-trained weights from ImageNet-21k...")
        
        if model_type == 'B16':
            vit_model = vit.vit_b16(
                image_size=self.img_height,
                activation='softmax',
                pretrained=True,
                include_top=False,
                pretrained_top=False
            )
            model_description = "ViT-Base/16 (12 layers, 768 hidden, 12 heads, ~86M params)"
        elif model_type == 'B32':
            vit_model = vit.vit_b32(
                image_size=self.img_height,
                activation='softmax',
                pretrained=True,
                include_top=False,
                pretrained_top=False
            )
            model_description = "ViT-Base/32 (12 layers, 768 hidden, 12 heads, ~88M params)"
        elif model_type == 'L16':
            vit_model = vit.vit_l16(
                image_size=self.img_height,
                activation='softmax',
                pretrained=True,
                include_top=False,
                pretrained_top=False
            )
            model_description = "ViT-Large/16 (24 layers, 1024 hidden, 16 heads, ~307M params)"
        elif model_type == 'L32':
            vit_model = vit.vit_l32(
                image_size=self.img_height,
                activation='softmax',
                pretrained=True,
                include_top=False,
                pretrained_top=False
            )
            model_description = "ViT-Large/32 (24 layers, 1024 hidden, 16 heads, ~307M params)"
        else:
            raise ValueError(f"Unknown model_type: {model_type}. Use 'B16', 'B32', 'L16', or 'L32'")
        
        print(f"✓ Model: {model_description}")
        
        # Freeze the base model initially
        vit_model.trainable = False
        
        # Build classification head
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        x = vit_model(inputs)
        
        # Classification head
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(512, activation='gelu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        x = layers.Dense(256, activation='gelu')(x)
        x = layers.Dropout(dropout_rate / 4)(x)
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        self.vit_base = vit_model
        
        print("\n✓ Pre-trained ViT model built successfully!")
        print(f"✓ Total parameters: {model.count_params():,}")
        print(f"✓ Base model frozen: {not vit_model.trainable}")
        print(f"✓ Pre-trained on: ImageNet-21k")
        print(f"✓ Model type: {model_type}")
        
        # Save architecture
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            f.write(f"Model: ViT-{model_type}\n")
            f.write(f"Description: {model_description}\n")
            f.write(f"Pre-trained: ImageNet-21k\n")
            f.write("="*70 + "\n\n")
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def unfreeze_and_fine_tune(self, learning_rate=0.00001):
        """Unfreeze ViT base for fine-tuning"""
        print(f"\n🔓 Unfreezing ViT base for fine-tuning...")
        
        self.vit_base.trainable = True
        
        # Recompile with much lower learning rate
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        print(f"✓ Fine-tuning enabled!")
        print(f"✓ Learning rate: {learning_rate}")
        print(f"✓ All transformer layers are now trainable")
    
    def train(self, epochs=50, early_stopping_patience=10):
        """Train the model"""
        print("\n" + "="*70)
        print(f"TRAINING {self.model_name}")
        print("="*70)
        
        if self.model is None:
            raise ValueError("Model not built. Call build_model() first.")
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-8,
                verbose=1
            ),
            keras.callbacks.CSVLogger(
                str(self.logs_dir / 'training_log.csv')
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Train model
        self.history = self.model.fit(
            self.train_dataset,
            epochs=epochs,
            validation_data=self.val_dataset,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"\n✓ Training completed!")
        print(f"⏱️  End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save model weights
        self.model.save_weights(str(self.model_dir / 'model_weights.h5'))
        print(f"✓ Model weights saved to: {self.model_dir / 'model_weights.h5'}")
        
        return self.history
    
    def evaluate(self):
        """Evaluate model on test set"""
        print("\n" + "="*70)
        print(f"EVALUATING {self.model_name}")
        print("="*70)
        
        print("\n📊 Evaluating on test set...")
        test_results = self.model.evaluate(self.test_dataset, verbose=1)
        
        print("\n✓ Test Results:")
        print(f"  Loss: {test_results[0]:.4f}")
        print(f"  Accuracy: {test_results[1]:.4f}")
        print(f"  Top-5 Accuracy: {test_results[2]:.4f}")
        
        # Get predictions
        print("\n📊 Generating detailed metrics...")
        y_true = []
        y_pred = []
        
        for images, labels in self.test_dataset:
            predictions = self.model.predict(images, verbose=0)
            y_true.extend(np.argmax(labels.numpy(), axis=1))
            y_pred.extend(np.argmax(predictions, axis=1))
        
        # Save results
        results = {
            'model_name': self.model_name,
            'test_loss': float(test_results[0]),
            'test_accuracy': float(test_results[1]),
            'test_top5_accuracy': float(test_results[2]),
            'num_classes': self.num_classes,
            'pretrained_on': 'ImageNet-21k',
            'evaluation_time': datetime.now().isoformat()
        }
        
        with open(self.logs_dir / 'test_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        # Generate confusion matrix
        self._plot_confusion_matrix(y_true, y_pred)
        
        # Generate classification report
        self._save_classification_report(y_true, y_pred)
        
        return results
    
    def _plot_confusion_matrix(self, y_true, y_pred):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(max(10, self.num_classes), max(8, self.num_classes * 0.8)))
        
        # Normalize confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        sns.heatmap(cm_normalized, annot=False, cmap='Blues', 
                    xticklabels=self.class_names if self.num_classes <= 20 else False,
                    yticklabels=self.class_names if self.num_classes <= 20 else False)
        
        plt.title(f'{self.model_name} - Confusion Matrix (Normalized)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Confusion matrix saved to: {self.plots_dir / 'confusion_matrix.png'}")
    
    def _save_classification_report(self, y_true, y_pred):
        """Save classification report"""
        report = classification_report(y_true, y_pred, target_names=self.class_names, 
                                      zero_division=0)
        
        with open(self.logs_dir / 'classification_report.txt', 'w') as f:
            f.write(f"{self.model_name} - Classification Report\n")
            f.write("="*70 + "\n\n")
            f.write(report)
        
        print(f"✓ Classification report saved to: {self.logs_dir / 'classification_report.txt'}")
    
    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            print("⚠️  No training history available")
            return
        
        print("\n📈 Generating training plots...")
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Plot accuracy
        axes[0].plot(self.history.history['accuracy'], label='Train Accuracy', linewidth=2)
        axes[0].plot(self.history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
        axes[0].set_title(f'{self.model_name} - Model Accuracy', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Plot loss
        axes[1].plot(self.history.history['loss'], label='Train Loss', linewidth=2)
        axes[1].plot(self.history.history['val_loss'], label='Val Loss', linewidth=2)
        axes[1].set_title(f'{self.model_name} - Model Loss', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'training_history.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Training plots saved to: {self.plots_dir / 'training_history.png'}")


# ============================================================================
# TRAINING FUNCTION
# ============================================================================

def train_pretrained_vit(dataset_path="final_merged_dataset", 
                         model_type='B16', 
                         epochs=50, 
                         batch_size=32):
    """
    Train pre-trained Vision Transformer
    
    Args:
        dataset_path: Path to dataset
        model_type: 'B16', 'B32', 'L16', 'L32'
        epochs: Number of training epochs
        batch_size: Batch size
    """
    print("\n\n" + "🔷"*35)
    print(f"TRAINING PRE-TRAINED VISION TRANSFORMER (ViT-{model_type})")
    print("🔷"*35)
    
    model = PretrainedViTDefectDetector(dataset_path)
    model.prepare_data(batch_size=batch_size)
    model.build_model(model_type=model_type, learning_rate=0.001)
    model.train(epochs=epochs)
    results = model.evaluate()
    model.plot_training_history()
    
    print("\n" + "="*70)
    print(f"🎉 PRE-TRAINED ViT-{model_type} TRAINING COMPLETED!")
    print("="*70)
    print(f"\n📊 Final Results:")
    print(f"  Test Accuracy: {results['test_accuracy']*100:.2f}%")
    print(f"  Test Loss: {results['test_loss']:.4f}")
    print(f"  Top-5 Accuracy: {results['test_top5_accuracy']*100:.2f}%")
    print(f"\n📁 All outputs saved to: ViT_B16_Pretrained_output/")
    
    return results




In [1]:
!pip install vit-keras


   ---------------------------------------- 0/2 [validators]
   ---------------------------------------- 2/2 [vit-keras]



In [5]:
train_pretrained_vit(model_type='B16')



🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷
TRAINING PRE-TRAINED VISION TRANSFORMER (ViT-B16)
🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷🔷

DATA PREPARATION - ViT-B16-Pretrained

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING PRE-TRAINED VISION TRANSFORMER (ViT-B16)

⏳ Downloading pre-trained weights from ImageNet-21k...
347502902/347502902 [==============================] - 197s 1us/step


c:\Users\User\.conda\envs\tf\lib\site-packages\vit_keras\utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


✓ Model: ViT-Base/16 (12 layers, 768 hidden, 12 heads, ~86M params)

✓ Pre-trained ViT model built successfully!
✓ Total parameters: 86,339,903
✓ Base model frozen: True
✓ Pre-trained on: ImageNet-21k
✓ Model type: B16

TRAINING ViT-B16-Pretrained

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-11-26 10:50:27

Epoch 1/50
215/215 [==============================] - 110s 464ms/step - loss: 1.4266 - accuracy: 0.6338 - top_5_accuracy: 0.8252 - val_loss: 1.0970 - val_accuracy: 0.6798 - val_top_5_accuracy: 0.8934 - lr: 0.0010
Epoch 2/50
215/215 [==============================] - 85s 394ms/step - loss: 1.0958 - accuracy: 0.6689 - top_5_accuracy: 0.8937 - val_loss: 0.9923 - val_accuracy: 0.6818 - val_top_5_accuracy: 0.9250 - lr: 0.0010
Epoch 3/50
215/215 [==============================] - 83s 386ms/step - loss: 1.0124 - accuracy: 0.6833 - top_5_accuracy: 0.8985 - val_loss: 0.9421 - val_accuracy: 0.7114 - val_top_5_accuracy: 0.9184 - lr: 0.0010
Epoch 4/50
215/215 [====================

ValueError: Number of classes, 61, does not match size of target_names, 63. Try specifying the labels parameter

In [ ]:
model = PretrainedViTDefectDetector("final_merged_dataset")
model.prepare_data(batch_size=32)
model.build_model(model_type='B16')
model.train(epochs=20)  # Initial training
model.unfreeze_and_fine_tune(learning_rate=0.00001)  # Unfreeze
model.train(epochs=10)  # Fine-tune
results = model.evaluate()


DATA PREPARATION - ViT-B16-Pretrained

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING PRE-TRAINED VISION TRANSFORMER (ViT-B16)

⏳ Downloading pre-trained weights from ImageNet-21k...


c:\Users\User\.conda\envs\tf\lib\site-packages\vit_keras\utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


✓ Model: ViT-Base/16 (12 layers, 768 hidden, 12 heads, ~86M params)

✓ Pre-trained ViT model built successfully!
✓ Total parameters: 86,339,903
✓ Base model frozen: True
✓ Pre-trained on: ImageNet-21k
✓ Model type: B16

TRAINING ViT-B16-Pretrained

🚀 Starting training for 20 epochs...
⏱️  Start time: 2025-11-26 11:43:51

Epoch 1/20
215/215 [==============================] - 75s 324ms/step - loss: 1.4211 - accuracy: 0.6342 - top_5_accuracy: 0.8235 - val_loss: 1.0840 - val_accuracy: 0.6823 - val_top_5_accuracy: 0.8934 - lr: 0.0010
Epoch 2/20
215/215 [==============================] - 69s 320ms/step - loss: 1.1080 - accuracy: 0.6711 - top_5_accuracy: 0.8913 - val_loss: 0.9550 - val_accuracy: 0.6940 - val_top_5_accuracy: 0.9199 - lr: 0.0010
Epoch 3/20
215/215 [==============================] - 70s 324ms/step - loss: 1.0103 - accuracy: 0.6861 - top_5_accuracy: 0.9060 - val_loss: 0.9486 - val_accuracy: 0.7129 - val_top_5_accuracy: 0.9179 - lr: 0.0010
Epoch 4/20
215/215 [=====================

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2, DenseNet121, DenseNet169, DenseNet201
from pathlib import Path
import json
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

class DefectDetectionModel:
    """Base class for defect detection models"""
    
    def __init__(self, dataset_path, model_name, img_height=224, img_width=224):
        self.dataset_path = Path(dataset_path)
        self.model_name = model_name
        self.img_height = img_height
        self.img_width = img_width
        self.model = None
        self.history = None
        self.class_names = []
        self.num_classes = 0
        
        # Create output directories
        self.output_dir = Path(f"{model_name}_output")
        self.model_dir = self.output_dir / "models"
        self.plots_dir = self.output_dir / "plots"
        self.logs_dir = self.output_dir / "logs"
        
        for dir_path in [self.output_dir, self.model_dir, self.plots_dir, self.logs_dir]:
            dir_path.mkdir(exist_ok=True)
    
    def prepare_data(self, validation_split=0.2, test_split=0.1, batch_size=32):
        """Prepare dataset with efficient TensorFlow pipeline"""
        print("\n" + "="*70)
        print(f"DATA PREPARATION - {self.model_name}")
        print("="*70)
        
        # Get all image paths and labels
        image_paths = []
        labels = []
        
        print("\n📁 Scanning dataset...")
        for class_dir in self.dataset_path.rglob("*"):
            if class_dir.is_dir():
                images = (list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + 
                         list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp")))
                
                if images:
                    class_name = class_dir.name
                    if class_name not in self.class_names:
                        self.class_names.append(class_name)
                    
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_name)
        
        self.class_names.sort()
        self.num_classes = len(self.class_names)
        print(f"✓ Found {len(image_paths)} images")
        print(f"✓ Number of classes: {self.num_classes}")
        print(f"✓ Classes: {self.class_names[:10]}{'...' if len(self.class_names) > 10 else ''}")
        
        # Create label mapping
        label_to_index = {label: idx for idx, label in enumerate(self.class_names)}
        labels_encoded = [label_to_index[label] for label in labels]
        
        # Create tf.data.Dataset
        dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels_encoded))
        dataset = dataset.shuffle(len(image_paths), seed=42)
        
        # Calculate splits
        train_size = int(len(image_paths) * (1 - validation_split - test_split))
        val_size = int(len(image_paths) * validation_split)
        test_size = len(image_paths) - train_size - val_size
        
        # Split datasets
        train_ds = dataset.take(train_size)
        val_ds = dataset.skip(train_size).take(val_size)
        test_ds = dataset.skip(train_size + val_size)
        
        # Process datasets
        self.train_dataset = train_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=True),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.val_dataset = val_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        self.test_dataset = test_ds.map(
            lambda x, y: self._load_and_preprocess(x, y, augment=False),
            num_parallel_calls=tf.data.AUTOTUNE
        ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        print(f"\n✓ Train samples: {train_size}")
        print(f"✓ Validation samples: {val_size}")
        print(f"✓ Test samples: {test_size}")
        
        return train_size, val_size, test_size
    
    def _load_and_preprocess(self, image_path, label, augment=False):
        """Load and preprocess image"""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, [self.img_height, self.img_width])
        
        # Data augmentation
        if augment:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.rot90(img, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
        
        img = tf.clip_by_value(img, 0.0, 255.0)
        img = img / 255.0  # Normalize to [0, 1]
        
        label_onehot = tf.one_hot(label, self.num_classes)
        return img, label_onehot
    
    def train(self, epochs=50, early_stopping_patience=10):
        """Train the model"""
        print("\n" + "="*70)
        print(f"TRAINING {self.model_name}")
        print("="*70)
        
        if self.model is None:
            raise ValueError("Model not built. Call build_model() first.")
        
        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=early_stopping_patience,
                restore_best_weights=True,
                verbose=1
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            ),
            keras.callbacks.CSVLogger(
                str(self.logs_dir / 'training_log.csv')
            )
        ]
        
        print(f"\n🚀 Starting training for {epochs} epochs...")
        print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Train model
        self.history = self.model.fit(
            self.train_dataset,
            epochs=epochs,
            validation_data=self.val_dataset,
            callbacks=callbacks,
            verbose=1
        )
        
        print(f"\n✓ Training completed!")
        print(f"⏱️  End time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save model weights
        self.model.save_weights(str(self.model_dir / 'model_weights.h5'))
        print(f"✓ Model weights saved to: {self.model_dir / 'model_weights.h5'}")
        
        return self.history
    
    def evaluate(self):
        """Evaluate model on test set"""
        print("\n" + "="*70)
        print(f"EVALUATING {self.model_name}")
        print("="*70)
        
        print("\n📊 Evaluating on test set...")
        test_results = self.model.evaluate(self.test_dataset, verbose=1)
        
        print("\n✓ Test Results:")
        print(f"  Loss: {test_results[0]:.4f}")
        print(f"  Accuracy: {test_results[1]:.4f}")
        if len(test_results) > 2:
            print(f"  Top-5 Accuracy: {test_results[2]:.4f}")
        
        # Get predictions for confusion matrix
        print("\n📊 Generating detailed metrics...")
        y_true = []
        y_pred = []
        
        for images, labels in self.test_dataset:
            predictions = self.model.predict(images, verbose=0)
            y_true.extend(np.argmax(labels.numpy(), axis=1))
            y_pred.extend(np.argmax(predictions, axis=1))
        
        # Save results
        results = {
            'test_loss': float(test_results[0]),
            'test_accuracy': float(test_results[1]),
            'num_classes': self.num_classes,
            'evaluation_time': datetime.now().isoformat()
        }
        
        if len(test_results) > 2:
            results['test_top5_accuracy'] = float(test_results[2])
        
        with open(self.logs_dir / 'test_results.json', 'w') as f:
            json.dump(results, f, indent=2)
        
        # Generate confusion matrix
        self._plot_confusion_matrix(y_true, y_pred)
        
        # Generate classification report
        self._save_classification_report(y_true, y_pred)
        
        return results
    
    def _plot_confusion_matrix(self, y_true, y_pred):
        """Plot confusion matrix"""
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(max(10, self.num_classes), max(8, self.num_classes * 0.8)))
        
        # Normalize confusion matrix
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        sns.heatmap(cm_normalized, annot=False, cmap='Blues', 
                    xticklabels=self.class_names if self.num_classes <= 20 else False,
                    yticklabels=self.class_names if self.num_classes <= 20 else False)
        
        plt.title(f'{self.model_name} - Confusion Matrix (Normalized)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Confusion matrix saved to: {self.plots_dir / 'confusion_matrix.png'}")
    
    def _save_classification_report(self, y_true, y_pred):
        """Save classification report"""
        unique_labels = sorted(list(set(y_true) | set(y_pred)))
        target_names_subset = [self.class_names[i] for i in unique_labels]
        
        report = classification_report(y_true, y_pred, 
                                      labels=unique_labels,
                                      target_names=target_names_subset, 
                                      zero_division=0)
        
        with open(self.logs_dir / 'classification_report.txt', 'w') as f:
            f.write(f"{self.model_name} - Classification Report\n")
            f.write("="*70 + "\n\n")
            f.write(f"Classes in test set: {len(unique_labels)}/{self.num_classes}\n\n")
            f.write(report)
        
        print(f"✓ Classification report saved to: {self.logs_dir / 'classification_report.txt'}")
    
    def plot_training_history(self):
        """Plot training history"""
        if self.history is None:
            print("⚠️  No training history available")
            return
        
        print("\n📈 Generating training plots...")
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        axes[0].plot(self.history.history['accuracy'], label='Train Accuracy', linewidth=2)
        axes[0].plot(self.history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
        axes[0].set_title(f'{self.model_name} - Model Accuracy', fontsize=14, fontweight='bold')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        axes[1].plot(self.history.history['loss'], label='Train Loss', linewidth=2)
        axes[1].plot(self.history.history['val_loss'], label='Val Loss', linewidth=2)
        axes[1].set_title(f'{self.model_name} - Model Loss', fontsize=14, fontweight='bold')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(self.plots_dir / 'training_history.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Training plots saved to: {self.plots_dir / 'training_history.png'}")


class MobileNetV2DefectDetector(DefectDetectionModel):
    """MobileNetV2 for defect detection - Lightweight and efficient"""
    
    def __init__(self, dataset_path, img_height=224, img_width=224):
        super().__init__(dataset_path, "MobileNetV2", img_height, img_width)
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.3, alpha=1.0):
        print("\n" + "="*70)
        print("BUILDING MOBILENETV2 MODEL")
        print("="*70)
        
        base_model = MobileNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(self.img_height, self.img_width, 3),
            alpha=alpha
        )
        
        base_model.trainable = False
        
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        x = layers.Rescaling(scale=2.0, offset=-1.0)(inputs)
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(dropout_rate / 2)(x)
        
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        with open(self.model_dir / 'model_architecture.txt', 'w') as f:
            f.write(f"Model: MobileNetV2 (alpha={alpha})\n")
            f.write("="*70 + "\n\n")
            model.summary(print_fn=lambda x: f.write(x + '\n'))
        
        return model
    
    def unfreeze_and_fine_tune(self, fine_tune_from_layer=100, learning_rate=0.0001):
        base_model = self.model.layers[2]
        base_model.trainable = True
        
        for layer in base_model.layers[:fine_tune_from_layer]:
            layer.trainable = False
        
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )


class DenseNetDefectDetector(DefectDetectionModel):
    """DenseNet for defect detection"""
    
    def __init__(self, dataset_path, model_variant='121', img_height=224, img_width=224):
        self.model_variant = model_variant
        super().__init__(dataset_path, f"DenseNet{model_variant}", img_height, img_width)
    
    def build_model(self, learning_rate=0.001, dropout_rate=0.3):
        print("\n" + "="*70)
        print(f"BUILDING DENSENET{self.model_variant} MODEL")
        print("="*70)
        
        if self.model_variant == '121':
            base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(self.img_height, self.img_width, 3))
        elif self.model_variant == '169':
            base_model = DenseNet169(include_top=False, weights='imagenet', input_shape=(self.img_height, self.img_width, 3))
        elif self.model_variant == '201':
            base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=(self.img_height, self.img_width, 3))
        else:
            raise ValueError("Variant must be 121, 169 or 201")
        
        base_model.trainable = False
        
        inputs = keras.Input(shape=(self.img_height, self.img_width, 3))
        x = tf.keras.applications.densenet.preprocess_input(inputs * 255.0)
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate/2)(x)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(dropout_rate/4)(x)
        
        outputs = layers.Dense(self.num_classes, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )
        
        self.model = model
        
        return model
    
    def unfreeze_and_fine_tune(self, fine_tune_from_layer=None, learning_rate=0.0001):
        defaults = {'121': 311, '169': 425, '201': 481}
        if fine_tune_from_layer is None:
            fine_tune_from_layer = defaults[self.model_variant]
        
        base_model = self.model.layers[2]
        base_model.trainable = True
        
        for layer in base_model.layers[:fine_tune_from_layer]:
            layer.trainable = False
        
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
        )


def train_mobilenetv2(dataset_path="final_merged_dataset", epochs=30, batch_size=32, alpha=1.0):
    model = MobileNetV2DefectDetector(dataset_path)
    model.prepare_data(batch_size=batch_size)
    model.build_model(learning_rate=0.001, alpha=alpha)
    model.train(epochs=epochs)
    results = model.evaluate()
    model.plot_training_history()
    return results


def train_densenet(dataset_path="final_merged_dataset", variant='121', epochs=30, batch_size=32):
    model = DenseNetDefectDetector(dataset_path, model_variant=variant)
    model.prepare_data(batch_size=batch_size)
    model.build_model(learning_rate=0.001)
    model.train(epochs=epochs)
    results = model.evaluate()
    model.plot_training_history()
    return results


In [20]:
train_mobilenetv2(dataset_path="final_merged_dataset", epochs=50, batch_size=32, alpha=1.0)



DATA PREPARATION - MobileNetV2

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING MOBILENETV2 MODEL

TRAINING MobileNetV2

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-11-27 22:51:38

Epoch 1/50
215/215 [==============================] - 73s 332ms/step - loss: 1.5692 - accuracy: 0.6221 - top_5_accuracy: 0.8023 - val_loss: 1.1267 - val_accuracy: 0.6690 - val_top_5_accuracy: 0.8924 - lr: 0.0010
Epoch 2/50
215/215 [==============================] - 57s 266ms/step - loss: 1.2241 - accuracy: 0.6524 - top_5_accuracy: 0.8756 - val_loss: 1.0294 - val_accuracy: 0.6854 - val_top_5_accuracy: 0.9128 - lr: 0.0010
Epoch 3/50
215/215 [==============================] - 37s 171ms/step - loss: 1.1126 - accuracy: 0.671

{'test_loss': 0.5133593082427979,
 'test_accuracy': 0.7983706593513489,
 'num_classes': 63,
 'evaluation_time': '2025-11-27T23:13:20.223068',
 'test_top5_accuracy': 0.9653767943382263}

In [21]:
train_densenet(dataset_path="final_merged_dataset", variant='121', epochs=50, batch_size=32)   


DATA PREPARATION - DenseNet121

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING DENSENET121 MODEL
29084464/29084464 [==============================] - 17s 1us/step

TRAINING DenseNet121

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-11-27 23:14:37

Epoch 1/50
215/215 [==============================] - 42s 181ms/step - loss: 1.5311 - accuracy: 0.6184 - top_5_accuracy: 0.8079 - val_loss: 1.2220 - val_accuracy: 0.6390 - val_top_5_accuracy: 0.8899 - lr: 0.0010
Epoch 2/50
215/215 [==============================] - 34s 158ms/step - loss: 1.1944 - accuracy: 0.6556 - top_5_accuracy: 0.8753 - val_loss: 1.0421 - val_accuracy: 0.6757 - val_top_5_accuracy: 0.8950 - lr: 0.0010
Epoch 3/50
215/215 [===============

{'test_loss': 0.47094088792800903,
 'test_accuracy': 0.8156822919845581,
 'num_classes': 63,
 'evaluation_time': '2025-11-27T23:37:43.492160',
 'test_top5_accuracy': 0.9725050926208496}

In [5]:
import tensorflow as tf
from tensorflow import keras

# Your MixupCutmixAugmenter class (keep as is)
class MixupCutmixAugmenter:
    def __init__(self, mixup_alpha=0.2, cutmix_alpha=1.0, prob=0.5):
        self.mixup_alpha = mixup_alpha
        self.cutmix_alpha = cutmix_alpha
        self.prob = prob

    def sample_beta(self, alpha):
        g1 = tf.random.gamma([], alpha, dtype=tf.float32)
        g2 = tf.random.gamma([], alpha, dtype=tf.float32)
        return g1 / (g1 + g2)

    def mixup(self, images, labels):
        lam = self.sample_beta(self.mixup_alpha)
        batch_size = tf.shape(images)[0]
        index = tf.random.shuffle(tf.range(batch_size))
        
        images2 = tf.gather(images, index)
        labels2 = tf.gather(labels, index)
        
        mixed_images = lam * images + (1 - lam) * images2
        mixed_labels = lam * labels + (1 - lam) * labels2
        
        return mixed_images, mixed_labels

    def cutmix(self, images, labels):
        batch = tf.shape(images)[0]
        H = tf.shape(images)[1]
        W = tf.shape(images)[2]
        
        index = tf.random.shuffle(tf.range(batch))
        images2 = tf.gather(images, index)
        labels2 = tf.gather(labels, index)
        
        lam = self.sample_beta(self.cutmix_alpha)
        cut_ratio = tf.sqrt(1 - lam)
        
        cut_w = tf.cast(tf.cast(W, tf.float32) * cut_ratio, tf.int32)
        cut_h = tf.cast(tf.cast(H, tf.float32) * cut_ratio, tf.int32)
        
        cx = tf.random.uniform([batch], 0, W, tf.int32)
        cy = tf.random.uniform([batch], 0, H, tf.int32)
        
        x1 = tf.clip_by_value(cx - cut_w // 2, 0, W)
        y1 = tf.clip_by_value(cy - cut_h // 2, 0, H)
        x2 = tf.clip_by_value(cx + cut_w // 2, 0, W)
        y2 = tf.clip_by_value(cy + cut_h // 2, 0, H)
        
        x_range = tf.tile(tf.reshape(tf.range(W), [1, 1, W]), [batch, H, 1])
        y_range = tf.tile(tf.reshape(tf.range(H), [1, H, 1]), [batch, 1, W])
        
        x_mask = tf.logical_and(
            x_range >= tf.reshape(x1, [-1, 1, 1]),
            x_range < tf.reshape(x2, [-1, 1, 1])
        )
        
        y_mask = tf.logical_and(
            y_range >= tf.reshape(y1, [-1, 1, 1]),
            y_range < tf.reshape(y2, [-1, 1, 1])
        )
        
        mask = tf.cast(tf.logical_and(x_mask, y_mask), tf.float32)
        mask = tf.expand_dims(mask, -1)
        
        new_images = images * (1 - mask) + images2 * mask
        
        box_area = tf.cast((x2 - x1) * (y2 - y1), tf.float32)
        lam_adj = 1 - (box_area / tf.cast(W * H, tf.float32))
        
        new_labels = lam_adj[:, None] * labels + (1 - lam_adj[:, None]) * labels2
        
        return new_images, new_labels

    def __call__(self, images, labels):
        if tf.random.uniform([]) > self.prob:
            return images, labels
        if tf.random.uniform([]) < 0.5:
            return self.mixup(images, labels)
        else:
            return self.cutmix(images, labels)


# CORRECTED TRAINING CODE
# Remove the debug line: eager_tensor = tf.constant([2.0896919, 2.1128857, 2.1081853])

# Step 1: Create model and prepare data
efficientnet = EfficientNetDefectDetector("final_merged_dataset")
efficientnet.prepare_data(batch_size=32)
efficientnet.build_model(learning_rate=0.001)

# Step 2: Apply Mixup/CutMix to training data
augmenter = MixupCutmixAugmenter(mixup_alpha=0.2, cutmix_alpha=1.0, prob=0.5)

# Apply augmentation - THIS IS THE KEY FIX
# Use .unbatch() then .batch() to ensure proper tensor handling
efficientnet.train_dataset = (
    efficientnet.train_dataset
    .unbatch()  # Unbatch first
    .batch(32)  # Rebatch
    .map(lambda x, y: augmenter(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

print("✓ Mixup/CutMix augmentation applied!")

# Step 3: Train with SIMPLIFIED callbacks to avoid serialization issues
class SimpleCSVLogger(keras.callbacks.Callback):
    """Custom CSV logger that handles tensors properly"""
    def __init__(self, filename):
        super().__init__()
        self.filename = filename
        self.file = None
        
    def on_train_begin(self, logs=None):
        self.file = open(self.filename, 'w')
        self.file.write('epoch,loss,accuracy,val_loss,val_accuracy\n')
        
    def on_epoch_end(self, epoch, logs=None):
        if logs:
            # Convert any tensors to Python floats
            loss = float(logs.get('loss', 0))
            acc = float(logs.get('accuracy', 0))
            val_loss = float(logs.get('val_loss', 0))
            val_acc = float(logs.get('val_accuracy', 0))
            
            self.file.write(f'{epoch},{loss},{acc},{val_loss},{val_acc}\n')
            self.file.flush()
    
    def on_train_end(self, logs=None):
        if self.file:
            self.file.close()

# Use simplified callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    SimpleCSVLogger('training_log.csv')  # Use custom logger
]

# Train
history = efficientnet.model.fit(
    efficientnet.train_dataset,
    epochs=50,
    validation_data=efficientnet.val_dataset,
    callbacks=callbacks,
    verbose=1
)

# Step 4: Evaluate
results = efficientnet.evaluate()
efficientnet.plot_training_history()

print(f"Test Accuracy: {results['test_accuracy']*100:.2f}%")


DATA PREPARATION - EfficientNetB0

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING EFFICIENTNET MODEL

✓ Model built successfully!
✓ Total parameters: 4,858,082
✓ Base model frozen: True
✓ Mixup/CutMix augmentation applied!
Epoch 1/50
215/215 [==============================] - 95s 430ms/step - loss: 1.9224 - accuracy: 0.5860 - top_5_accuracy: 0.7503 - val_loss: 1.4944 - val_accuracy: 0.6548 - val_top_5_accuracy: 0.8348 - lr: 0.0010
Epoch 2/50
215/215 [==============================] - 50s 232ms/step - loss: 1.5709 - accuracy: 0.6259 - top_5_accuracy: 0.8242 - val_loss: 1.0848 - val_accuracy: 0.6849 - val_top_5_accuracy: 0.8924 - lr: 0.0010
Epoch 3/50
215/215 [==============================] - 54s 250ms/step - l

In [6]:
def prepare_data_with_advanced_augmentation(
    dataset_path,
    class_names,
    num_classes,
    img_height=224,
    img_width=224,
    validation_split=0.2,
    test_split=0.1,
    batch_size=32,
    use_mixup_cutmix=True,
    use_autoaugment=True
):
    """
    Prepare data with advanced augmentation
    
    DROP-IN REPLACEMENT for your existing prepare_data() function!
    
    Args:
        dataset_path: Path to dataset
        class_names: List of class names
        num_classes: Number of classes
        img_height: Image height
        img_width: Image width
        validation_split: Validation split ratio
        test_split: Test split ratio
        batch_size: Batch size
        use_mixup_cutmix: Enable Mixup/CutMix (recommended!)
        use_autoaugment: Enable AutoAugment (recommended!)
    
    Returns:
        train_dataset, val_dataset, test_dataset
    """
    from pathlib import Path
    
    dataset_path = Path(dataset_path)
    
    # Get all image paths and labels
    image_paths = []
    labels = []
    
    print("\n📁 Scanning dataset...")
    for class_dir in dataset_path.rglob("*"):
        if class_dir.is_dir():
            images = (list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png")) + 
                     list(class_dir.glob("*.jpeg")) + list(class_dir.glob("*.bmp")))
            
            if images:
                class_name = class_dir.name
                if class_name in class_names:
                    class_idx = class_names.index(class_name)
                    for img_path in images:
                        image_paths.append(str(img_path))
                        labels.append(class_idx)
    
    print(f"✓ Found {len(image_paths)} images")
    
    # Create dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.shuffle(len(image_paths), seed=42)
    
    # Calculate splits
    train_size = int(len(image_paths) * (1 - validation_split - test_split))
    val_size = int(len(image_paths) * validation_split)
    
    # Split datasets
    train_ds = dataset.take(train_size)
    val_ds = dataset.skip(train_size).take(val_size)
    test_ds = dataset.skip(train_size + val_size)
    
    # Load and preprocess function
    def load_and_preprocess(image_path, label, augment=False):
        img = tf.io.read_file(image_path)
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, [img_height, img_width])
        
        # Basic augmentation (only for training)
        if augment and not use_autoaugment:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_flip_up_down(img)
            img = tf.image.random_brightness(img, max_delta=0.2)
            img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
            img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
            img = tf.image.rot90(img, k=tf.random.uniform([], 0, 4, dtype=tf.int32))
        
        img = tf.clip_by_value(img, 0.0, 255.0)
        img = img / 255.0  # Normalize to [0, 1]
        
        # AutoAugment (only for training)
        if augment and use_autoaugment:
            img = autoaugment_policy(img)
        
        label_onehot = tf.one_hot(label, num_classes)
        return img, label_onehot
    
    # Process datasets
    train_dataset = train_ds.map(
        lambda x, y: load_and_preprocess(x, y, augment=True),
        num_parallel_calls=tf.data.AUTOTUNE
    ).batch(batch_size)
    
    val_dataset = val_ds.map(
        lambda x, y: load_and_preprocess(x, y, augment=False),
        num_parallel_calls=tf.data.AUTOTUNE
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    test_dataset = test_ds.map(
        lambda x, y: load_and_preprocess(x, y, augment=False),
        num_parallel_calls=tf.data.AUTOTUNE
    ).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    # Apply Mixup/CutMix to training data
    if use_mixup_cutmix:
        print("✓ Applying Mixup/CutMix augmentation")
        augmenter = MixupCutmixAugmenter(mixup_alpha=0.2, cutmix_alpha=1.0, prob=0.5)
        train_dataset = train_dataset.map(
            lambda x, y: augmenter(x, y),
            num_parallel_calls=tf.data.AUTOTUNE
        )
    
    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
    
    print(f"✓ Train samples: {train_size}")
    print(f"✓ Validation samples: {val_size}")
    print(f"✓ Test samples: {len(image_paths) - train_size - val_size}")
    print(f"✓ Advanced augmentation enabled!")
    
    return train_dataset, val_dataset, test_dataset


In [7]:
# Simple augmentation without AutoAugment (works perfectly!)

# Step 1: Create model and prepare data normally
efficientnet = EfficientNetDefectDetector("final_merged_dataset")
efficientnet.prepare_data(batch_size=32)
efficientnet.build_model(learning_rate=0.001)

# Step 2: Apply Mixup/CutMix to training data
from tensorflow import keras

augmenter = MixupCutmixAugmenter(mixup_alpha=0.2, cutmix_alpha=1.0, prob=0.5)

# Apply augmentation to training dataset
efficientnet.train_dataset = efficientnet.train_dataset.map(
    lambda x, y: augmenter(x, y),
    num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

print("✓ Mixup/CutMix augmentation applied!")

# Step 3: Train
efficientnet.train(epochs=50)

# Step 4: Evaluate
results = efficientnet.evaluate()
efficientnet.plot_training_history()

print(f"Test Accuracy: {results['test_accuracy']*100:.2f}%")


DATA PREPARATION - EfficientNetB0

📁 Scanning dataset...
✓ Found 9808 images
✓ Number of classes: 63
✓ Classes: ['Missing_hole', 'Missing_hole_rotation', 'Mouse_bite', 'Mouse_bite_rotation', 'Open_circuit', 'Open_circuit_rotation', 'PCB_USED', 'Short', 'Short_rotation', 'Spur']...

✓ Train samples: 6865
✓ Validation samples: 1961
✓ Test samples: 982

BUILDING EFFICIENTNET MODEL

✓ Model built successfully!
✓ Total parameters: 4,858,082
✓ Base model frozen: True
✓ Mixup/CutMix augmentation applied!

TRAINING EfficientNetB0

🚀 Starting training for 50 epochs...
⏱️  Start time: 2025-12-07 14:03:51

Epoch 1/50
215/215 [==============================] - 40s 173ms/step - loss: 1.8065 - accuracy: 0.5959 - top_5_accuracy: 0.7633 - val_loss: 1.3921 - val_accuracy: 0.6400 - val_top_5_accuracy: 0.8587 - lr: 0.0010
Epoch 2/50
210/215 [============================>.] - ETA: 0s - loss: 1.5920 - accuracy: 0.6210 - top_5_accuracy: 0.8210

KeyboardInterrupt: 